In [ ]:
'''
TODO

- nel calcolo del diametro karst per le trincee considerare la distanza tra m_s-l_m e bedrock (h_quat+h_perm-h_rilevato)
- introdurre una distribuzione exponenziale troncata
- nel calcolo a ritroso dividere le subsidenze dagli inbuti
- la def funziona per gli inbuti, fare una nuova def per le subsidenze
- capire come gestire quegli inbuti che dal calcolo a ritroso non hanno soluzione (ex coperture alte)
- cambiare nome h_rilevato con h_livelletta
- in df0 dbase (contatto roccia) dave (superficie)
- in df_zo dbase(superficie) dave(contatto roccia)
- estrarre il num_karst medio in profondità (ad esempio per i viadotti e per stimare i riempimenti con il grout)

'''

In [ ]:
#!/usr/bin/env conda run -n newenv3 python
import sys
print(sys.executable)    ### C:\ProgramData\Anaconda3_1\envs\newenv3\python.exe
#!{sys.executable} -m pip install tables              ##  df.to_hdf

from platform import python_version
print('The python_version is: ',python_version())

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
!jupyter kernelspec list
# pip install xlrd
# python -m pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose

## Note

- Non può prevedere le caverne e gli eventi con probabilità bassa
- L'assunzione principale si basa sul fatto che:
        - i diametri di karst piccoli hanno una probabilità alta
        - i diametri di karst grandi hanno una prob. bassa
        - da qui l'assunzione di utilizzare una prob exp troncata ad un valore massimo

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from descartes import PolygonPatch
import math
from ipyleaflet import Map, GeoJSON
import geojson
import numpy as np
#from mpl_toolkits.basemap import Basemap
from ipyleaflet import Map
import mpld3
from dbfread import DBF
mpld3.enable_notebook()
#to import the basemap library give the direct path to the library
# import os
# os.environ["PROJ_LIB"]="C:\\Users\\Anaconda3\\Library\\share"
import pandas as pd
from simpledbf import Dbf5
import math
from Finddistribution import *
import numpy as np

from scipy.stats import expon
import matplotlib.pyplot as plt

import os
import types

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


import plotly.graph_objects as go
from scipy.interpolate import griddata
from pyproj import Proj
import plotly.express as px

import random
from tqdm import tqdm

import sys
from tqdm import trange
import seaborn as sns

from IPython.display import display
pd.options.display.max_columns = None

from scipy.stats import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import truncexpon

import scipy.spatial as spatial
from scipy.interpolate import griddata
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from scipy.spatial import ConvexHull, convex_hull_plot_2d

from matplotlib.patches import Polygon

%matplotlib inline

#### Variabili globali

In [ ]:
# limite di altezza per distinguere tra viadotti alti e bridge bassi
lim_via_bridge = 12.5

vitautile = 100

# soglia_dia_karst per differenziare karst small da karst grandi
soglia_dia_karst = 3
# 1000 numero di simulazioni stochastiche
epochs = 100
# metri in cui è diviso il file principale in punti "Tabella_completa_karst2_3.csv"
m_punti = 20
#109 serve per la routine principale
pkfinale = 40
# Groupby Accorpa in settori se uso: 50 ogni km, 5 ogni 100m
div_settori = 50
# correzione distr exp per aumentare stima diametri bassi
# questo perché in teoria la distribuzione è exp troncata
correzione_distr_exp = 2
# soglia minima per fare la distribuzione esponenziale
n_karst_soglia = 40
# size per la distribuzione esponenziale
size_exp = 1000
##

### CHECK CONGRUENZA FILE INPUT

In [ ]:
df_prop = pd.read_csv('df_prop.csv', sep=';', index_col=0)
df_prop = df_prop.apply(pd.to_numeric, errors='ignore', downcast = 'float')
df_prop.set_index('IGE_princi', inplace=True)

if '3a' in df_prop.index: 
    print('OK: colonna index impostata correttamente come "IGE_princi"')
else:
    print('ERROR: colonna index potrebbe non avere gli IGE_princi')

list_ige = ['3', '3a', '12', '6', '2a-1', '5-1', '2', '9', '6a', '6-1', '13', '11',
       '4', '13-1', '17', '16', '19', '7', '18', '15a', '14', '340', '14-1',
       '14-2', '14']
err = 0
for i in list_ige:
    if i not in df_prop.index.tolist():
        print("IGE ",i,"ERROR: non è presente nella precedente lista di IGE")
        err = 1
    else:
        pass
if err==0: print("OK: Tutti i nuovi 'IGE' erano presenti nella lista precedente")




# index nome IGE il resto del dataframe è tutto numerico
df_prop_rock = pd.read_csv('df_prop_rock.csv', index_col=0)
df_prop_rock = df_prop_rock.apply(pd.to_numeric, errors='coerce', downcast = 'float')

if 'Gessi' in df_prop_rock.index: 
    print('OK: La colonna index del df_prop_rock è correttamente impostata sul tipo di roccia')
else:
    print('ERROR: La colonna index del df_prop_rock non è correttamente impostata sul tipo di roccia')


df_orig = pd.read_csv('Tabella_completa_karst2_3.csv', sep=';', index_col=0)

err = 0
for i in df_orig.IGE.tolist():
    if i not in df_prop.index.tolist():
        print('ERROR: non corrisponde l IGE: ', i)
        err += 1
for i in df_orig.IGE_sec.tolist():
    if i not in df_prop.index.tolist():
        print('ERROR: non corrisponde l IGE: ', i)
        err += 1
if err == 0 : print('OK: Tutti gli IGE nel dataframe principale sono contenuti nel dataframe prop_list')
    
if df_prop.shape[0] == len(df_prop.index.unique().tolist()):
    print('OK le IGE non sono ripetute sono univoche nel file df_prop')
else:
    print('ERROR gli IGE non sono univoci nel file df_prop')
    
    
### df0 =======================================================================================

df0 = pd.read_csv('kast-csv2.csv', sep=";")

err = 0
for i in ['long', 'lat', 'type', 'dave', 'era']:
    if i in df0.columns.tolist():
        pass
    else:
        print('ERROR The column -> ',i,' in df0 do not exists!')
        err +=1
if err==0: print('OK checked columns in df0')

if df0.type.unique().tolist() == ['Inbuto', 'Subsidenza']: 
    print('OK checked column type of df0')
else:
    print('ERROR check column type of df0')
    


### INIZIO

In [ ]:
df_orig = pd.read_csv('Tabella_completa_karst2_3.csv', sep=';', index_col=0)

In [ ]:
# controlla quali colonne sono stringhe

df_orig.select_dtypes('object').columns

In [ ]:
# converti colonne che devono essere numeriche non necessario se la riga sopra fornisce un risultato corretto4

col_list = ['Road_level','m_s_l_m_','calc_top','gessi_top','marne_top','Arg_perm','Arg_giura','h_perm','h_calc','h_gessi','h_giur','h_quat','h_marne']

for col in col_list:
    df_orig[col] = round(df_orig[col].astype('float'),1)

In [ ]:
df_orig['h_livelletta'] = df_orig['Road_level'] - df_orig['m_s_l_m_']

#### Creo df_orig

In [ ]:
df_orig.columns

In [ ]:
# df = df_orig[['IGE', 'Litho', 'IGE_sec', 'IGE_SEC_PERC', 'LITHO_SEC', 'POINT_X', 'POINT_Y','h_quat','Tipo','Tipo_plus','h_rilevato']].copy()
df = df_orig.copy()

In [ ]:
# # se non trova la colonna Tipo_plus usare questo script
# df_temp_tipo = pd.read_csv('df_tipo_plus.csv', index_col=0)
# df['Tipo_plus'] = df_temp_tipo['Tipo_plus']

In [ ]:
df['h_perm'] = df_orig['h_perm'] + df_orig['h_giur'] + df_orig['h_marne']

In [ ]:
def f(row):
    if row['gessi_top'] > row['calc_top']:
        val = 'g'
    else:
        val = 'c'
    return val

df['bedrock'] = df_orig.apply(f, axis=1)

#### Correggo gli spessori negativi per errore precisione Leader / profilo GEO

In [ ]:
df.loc[df['h_quat']<0,['h_quat']] = 0
df.loc[df['h_perm']<0,['h_perm']] = 0

#### Carico file quote piano campagna

In [ ]:
df_topography_lidar = pd.read_csv("Topography_Lidar_10m_vertices.csv")

In [ ]:
df_topography_SRTM = pd.read_csv("SRTM_Karst_AREA vertices.csv")

#### Creo Dataframe con le proprietà meccaniche  "df_prop" "df_prop_rock"

In [ ]:
df_prop = pd.read_csv('df_prop.csv', sep=';', index_col=0)

In [ ]:
#### Creo un dataframe con le proprietà meccaniche

# index nome IGE il resto del dataframe è tutto numerico
df_prop = pd.read_csv('df_prop.csv', sep=';', index_col=0)
df_prop = df_prop.apply(pd.to_numeric, errors='ignore', downcast = 'float')
df_prop.set_index('IGE_princi', inplace=True)


# index nome IGE il resto del dataframe è tutto numerico
df_prop_rock = pd.read_csv('df_prop_rock.csv', index_col=0)
df_prop_rock = df_prop_rock.apply(pd.to_numeric, errors='coerce', downcast = 'float')

In [ ]:
col_prop_base = [i if (("MAX" not in i)&("MIN" not in i)&("stdev" not in i)) else "" for i in df_prop.columns.to_list()]
col_prop_base = list(filter(None, col_prop_base))

In [ ]:
# litotipi associabili al permiano
df_prop.loc[['14','16','17'],col_prop_base]

In [ ]:
# alcune colonne importanti continuano ad essere stringe per la presenza di cambi N/A di excel

In [ ]:
df_prop.select_dtypes('object').columns

In [ ]:
df_prop_rock.select_dtypes('object').columns

### Creo colonna progressive e gestisco punti tracciato

In [ ]:
df['POINT_X_Diff'] = df.POINT_X.diff().replace(np.nan, 0)
df['POINT_Y_Diff'] = df.POINT_Y.diff().replace(np.nan, 0)
df['length'] = np.sqrt(df.POINT_X_Diff**2 + df.POINT_Y_Diff**2).round().astype('int')
df['progressiva'] = df.length.cumsum().round().astype('int')

# correggo la prima lunghezza posso farlo dopo aver creato la colonna 'progressiva'
df.loc[0, 'length'] = 20

# verifico le colonne stringhe
df.select_dtypes('object').columns

# calcololo offset del tracciato

### creo colonna con differenza ogni 2 punti

In [ ]:
df['diff_X_roll2'] = df.POINT_X.diff(periods=2).shift(-1).fillna(method = 'bfill').fillna(method = 'ffill')
df['diff_Y_roll2'] = df.POINT_Y.diff(periods=2).shift(-1).fillna(method = 'bfill').fillna(method = 'ffill')

In [ ]:
df1 = df[[
 'POINT_X',
 'POINT_Y',
 'diff_X_roll2',
 'diff_Y_roll2']]

In [ ]:
df1.columns

In [ ]:
def perpendicular(a) :
    b = np.empty_like(a)
    b[0] = a[1]
    b[1] = -a[0]
    return b

def calcangle_2v(x):
    v2 = np.array([0, 1])
    v1 = x/np.linalg.norm(x)
    dot_prod = np.dot(v1, v2)
    return math.degrees(np.arccos(dot_prod))


def offset(cx, cy, vx, vy, offset):

    vdiff = np.array([vx, vy]).T
    
    # normalize the vector difference
    # vdiffnorm = vdiff/(np.linalg.norm(vdiff))
    # for row wise normalization need to set axis = 1 then to avoid errors on shape just use np.newaxis
    vdiffnorm = vdiff/(np.linalg.norm(vdiff, axis=1))[:, np.newaxis]
    # calculate its normalized perpendicular vector
    
    vnorm = np.apply_along_axis(perpendicular, 1, vdiffnorm)
    angle = np.apply_along_axis(calcangle_2v, 1, vnorm)
    voff = vnorm * offset
    
    vtrack = np.array([cx,cy]).T
    vtrackup = vtrack + voff
    vtrackdown = vtrack - voff
    
    #breakpoint()
    
    return vtrackup, vtrackdown, angle
    

In [ ]:
vtrackup, vtrackdown, angle = offset(df.POINT_X, df.POINT_Y, df1.diff_X_roll2, df1.diff_Y_roll2, 100 )
vtrackup500, vtrackdown500, angle500 = offset(df.POINT_X, df.POINT_Y, df1.diff_X_roll2, df1.diff_Y_roll2, 500 )
vtrackup2000, vtrackdown2000, angle2000 = offset(df.POINT_X, df.POINT_Y, df1.diff_X_roll2, df1.diff_Y_roll2, 2500 )

#### Creo colonne trackup , trackdown, 2000 500 100 nel df    (2000 è 2500m above below tracciato)

In [ ]:
columns_ = ['vtrackupx','vtrackdownx','vtrackup500x','vtrackdown500x','vtrackup2000x','vtrackdown2000x']
values_ = [vtrackup[:,0],vtrackdown[:,0],vtrackup500[:,0],vtrackdown500[:,0],vtrackup2000[:,0],vtrackdown2000[:,0]]
for e,i in enumerate(columns_):
    df[i] = values_[e]
columns_ = ['vtrackupy','vtrackdowny','vtrackup500y','vtrackdown500y','vtrackup2000y','vtrackdown2000y']
values_ = [vtrackup[:,1],vtrackdown[:,1],vtrackup500[:,1],vtrackdown500[:,1],vtrackup2000[:,1],vtrackdown2000[:,1]] 
for e,i in enumerate(columns_):
    df[i] = values_[e]

##### Creo le colonne utili per il calcolo delle contromisure

In [ ]:
# One hot encoding for Tipe (rilevato, trincea, viadotto)
df = pd.concat((df, pd.get_dummies(df['Tipo'])), axis=1)
df = df.loc[:,~df.columns.duplicated()]

In [ ]:
df['h_rilevato_true'] = np.where((df.Tipo == 'Rilevato'), df['h_rilevato'],np.nan)
df['h_trincea'] = np.where((df.Tipo == 'Trincea'), df['h_rilevato'],np.nan)
df['h_viadotto'] = np.where(((df.Tipo != 'Trincea')&(df.Tipo != 'Rilevato')), df['h_rilevato'],np.nan)
df['h_copertura'] = df.h_quat + df.h_perm
df['h_copertura_ril'] = np.where((df.Tipo == 'Rilevato'), df['h_copertura'],np.nan)
df['h_copertura_tri'] = np.where((df.Tipo == 'Trincea'), (df['h_copertura']+df['h_rilevato']),np.nan)
df['h_copertura_via'] = np.where(((df.Tipo != 'Trincea')&(df.Tipo != 'Rilevato')), df['h_copertura'],np.nan)

df['Viaduct'] = np.where(df['h_viadotto'] >= lim_via_bridge, df["Ponte/viadotto new"], np.nan)
df['Bridge'] = np.where(df['h_viadotto'] < lim_via_bridge, df["Ponte/viadotto new"], np.nan)  
df['LRilevato'] = df['Rilevato']*df['length']
df['LTrincea'] = df['Trincea']*df['length']
df['LViaduct'] = df['Viaduct']*df['length']
df['LBridge'] = df['Bridge']*df['length']

In [ ]:
df[(df.progressiva>=21000)&(df.progressiva<22000)]['h_copertura_tri'].mean()

#### Pulisco colonne inutili

In [ ]:
df.drop(columns=['POINT_X_Diff','POINT_Y_Diff','diff_X_roll2','diff_Y_roll2'], inplace=True, errors='ignore')

### Carico il file con i Karst

In [ ]:
# df0 = pd.read_csv('karst.csv')
df0 = pd.read_csv('kast-csv2.csv', sep=";")
# df0.to_csv('kast-csv2.csv', sep=";")
df0.select_dtypes('object').columns

In [ ]:
df0.head(2)

### CREO df_g50 --> df (ogni 20m) condensato ogni 1km

In [ ]:
# perché la media sia efficace bisogna convertire lo zero con np.nan dove ci sono delle colonne parziali
# Attenzione quando condensi anche sul h_perm

df['h_rilevato_true'] = df['h_rilevato_true'].replace(0, np.NaN)
df['h_trincea'] = df['h_trincea'].replace(0, np.NaN)
df['h_viadotto'] = df['h_viadotto'].replace(0, np.NaN)
df['h_copertura_ril'] = df['h_copertura_ril'].replace(0, np.NaN)
df['h_copertura_tri'] = df['h_copertura_tri'].replace(0, np.NaN)
df['h_copertura_via'] = df['h_copertura_via'].replace(0, np.NaN)

df_g50 = df.groupby(df.index // div_settori).agg({'progressiva':'first', 'POINT_X':'first', 'POINT_Y':'first', 'IGE':max, 'Litho':max, 'h_quat':'mean', 'h_perm':'mean',
                                         'vtrackupx':'first', 'vtrackupy':'first', 'vtrackdownx':'first', 'vtrackdowny':'first',
                                         'vtrackup500x':'first', 'vtrackup500y':'first', 'vtrackdown500x':'first', 'vtrackdown500y':'first',
                                         'vtrackup2000x':'first', 'vtrackup2000y':'first', 'vtrackdown2000x':'first', 'vtrackdown2000y':'first',
                                         'length':'sum','h_rilevato':'mean','h_rilevato_true':'mean','h_trincea':'mean','h_viadotto':'max',
                                         'LRilevato':'sum','LTrincea':'sum','LViaduct':'sum','LBridge':'sum',
                                         'h_copertura':'mean', 'h_copertura_ril':'mean', 'h_copertura_tri':'mean', 'h_copertura_via':'mean', 'Tipo_plus':'max'})
# riaggiungo l'ultima riga

df_g50 = df_g50.append(df[df_g50.columns.tolist()].tail(1), ignore_index=True, sort=False)

# ripristino gli zeri sono obbligato perché il df lo uso per le interpolazioni
df.fillna(0, inplace=True)

In [ ]:
df_g50.loc[[21]]

In [ ]:
# df_g50 = pd.read_csv('df_g50.csv', index_col=0, sep=';')

### CONTO I KARST CHE CADONO NELL'AREA DI 1KM 
####  - aggiorno il df0 con i carst con la colonna "sector2000_50" che associa al karst un settore g50 di riferimento
####  - aggiorno il df_g50 con le colonne 'n_karst' e 'sector_area'

In [ ]:
df_g50['n_karst'] = np.nan
df_g50['sector_area'] = np.nan
df0['sector2000_50'] = np.nan
df0['check_100'] = False
df0['check_500'] = False

for i in range(df_g50.shape[0]-1):
    #######        prima escludevo tutti i punti oltre la pk 60000 adesso oltre la pk 110000
    if (df_g50.progressiva[i] > 110000):
        pass
    
    else:
        polygon = Polygon([tuple(df_g50[['vtrackup2000x','vtrackup2000y']].loc[i,:]),
                          tuple(df_g50[['vtrackup2000x','vtrackup2000y']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdown2000x','vtrackdown2000y']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdown2000x','vtrackdown2000y']].loc[i,:])])
        polygon2 = Polygon([tuple(df_g50[['vtrackupx','vtrackupy']].loc[i,:]),
                          tuple(df_g50[['vtrackupx','vtrackupy']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdownx','vtrackdowny']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdownx','vtrackdowny']].loc[i,:])])
        polygon3 = Polygon([tuple(df_g50[['vtrackup500x','vtrackup500y']].loc[i,:]),
                          tuple(df_g50[['vtrackup500x','vtrackup500y']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdown500x','vtrackdown500y']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdown500x','vtrackdown500y']].loc[i,:])])
        n_karst = 0
        
        # ogni volta itero sul n° di karst di df0
        # TODO : per ottimizzare se finisce in un settore rimuovo la riga dal dataframe
        
        for e in range(df0.shape[0]):
            point = Point(df0.loc[df0.index[e],['long']], df0.loc[df0.index[e],['lat']])
            if polygon.contains(point):
                df0.loc[df0.index[e], 'sector2000_50'] = i
                n_karst += 1
            else:
                pass
            if polygon.contains(point): df0.loc[df0.index[e], 'check_2000'] = True
            if polygon2.contains(point): df0.loc[df0.index[e], 'check_100'] = True
            if polygon3.contains(point): df0.loc[df0.index[e], 'check_500'] = True
        
        df_g50.loc[df_g50.index[i], 'n_karst'] = n_karst
        # area in km^2
        df_g50.loc[df_g50.index[i], 'sector_area'] = polygon.area/1e6
        df_g50.loc[df_g50.index[i], 'sector_area_100'] = polygon2.area/1e6
        df_g50.loc[df_g50.index[i], 'sector_area_500'] = polygon3.area/1e6

In [ ]:
df_g50['n_karst'] = df_g50['n_karst'].fillna(0)
df_g50['sector_area'] = df_g50['sector_area'].fillna(method = 'ffill')   # oltre la pk50 tutto uguale

In [ ]:
df_g50['n_karst_det'] = df_g50['n_karst']

#### Drop karst row outside 2500m boundary  (recall that sector2000 is 2500m)

In [ ]:
# rimuovo i punti che finiscono fuori settore
df0.dropna(subset=['sector2000_50'], inplace=True)
df0['check_500'] = df0.check_500.fillna(False)
df0['check_100'] = df0.check_100.fillna(False)

In [ ]:
df0['sector2000_50'] = df0.sector2000_50.astype('float')
df0['sector2000_50_str'] = df0.sector2000_50.astype('str')

In [ ]:
### Creo colonna con gli indici dei karst  "sector2000_50" inizialmente è nell'indice ma la sposto in una colonna

df_g50.reset_index(inplace = True)
df_g50.rename(columns={'index':'sector2000_50'}, inplace = True)

In [ ]:
df_g50.to_csv('df_g50.csv', sep=";")

### CREO LA FUNZIONE PER CALCOLARE DIA KARST

#### Creo le funzioni di interpolazione per associare ad ogni karst di df0 delle proprieta: progressiva, h_rilevato, Tipo, IGE, h_perm ect

In [ ]:
df_pred = df[['POINT_X','POINT_Y','h_perm','h_quat','IGE', 'IGE_sec', 'IGE_SEC_PERC','Tipo', 'Tipo_plus', 'h_rilevato_true', 'h_trincea', 
              'h_viadotto', 'h_copertura_tri', 'h_copertura_ril', 'h_copertura', 'h_copertura_via', 'h_rilevato', 'progressiva']].append(
    df[['vtrackup2000x','vtrackup2000y','h_perm','h_quat','IGE', 'IGE_sec', 'IGE_SEC_PERC','Tipo', 'Tipo_plus', 'h_rilevato_true', 'h_trincea', 
        'h_viadotto', 'h_copertura_tri', 'h_copertura_ril', 'h_copertura', 'h_copertura_via', 'h_rilevato', 'progressiva']].rename(columns={'vtrackup2000x':'POINT_X','vtrackup2000y':'POINT_Y'}), 
    ignore_index=True).append(
    df[['vtrackdown2000x','vtrackdown2000y','h_perm','h_quat','IGE', 'IGE_sec', 'IGE_SEC_PERC','Tipo', 'Tipo_plus', 'h_rilevato_true', 'h_trincea', 
        'h_viadotto', 'h_copertura_tri', 'h_copertura_ril', 'h_copertura', 'h_copertura_via', 'h_rilevato', 'progressiva']].rename(columns={'vtrackdown2000x':'POINT_X','vtrackdown2000y':'POINT_Y'}), 
    ignore_index=True)

In [ ]:
def predict(points,values,points_pred):
    '''
    input
     points = numpy.array(i,2)
     values = numpy array(i,)
     points_pred = numpy.array(i,2)
    output
     predicted_values 
    '''
    points = np.array(points)
    values = np.array(values)
    points_pred = np.array(points_pred)
    if (type(values[0].item()) == float) | (type(values[0].item()) == int):
        values_pred = griddata(points, values, points_pred, method='linear')
    else:
        values_pred = griddata(points, values, points_pred, method='nearest')
    
    return values_pred

In [ ]:
df.shape, df_orig.shape, df_pred.shape

In [ ]:
df_pred.head(3)

In [ ]:
points = df_pred.loc[:,['POINT_X','POINT_Y']]
values_h_perm = df_pred.loc[:,['h_perm']]
values_h_quat = df_pred.loc[:,['h_quat']]
values_ige = df_pred.loc[:,['IGE']]
values_ige_sec = df_pred.loc[:,['IGE_sec']]
values_ige_sec_perc = df_pred.loc[:,['IGE_SEC_PERC']]
values_tipo = df_pred.loc[:,['Tipo']]
values_tipo_plus = df_pred.loc[:,['Tipo_plus']]

values_h_rilevato = df_pred.loc[:,['h_rilevato']]
values_h_rilevato_true = df_pred.loc[:,['h_rilevato_true']]
values_h_trincea = df_pred.loc[:,['h_trincea']]
values_h_viadotto = df_pred.loc[:,['h_viadotto']]

values_h_copertura = df_pred.loc[:,['h_copertura']]
values_h_copertura_ril = df_pred.loc[:,['h_copertura_ril']]
values_h_copertura_tri = df_pred.loc[:,['h_copertura_tri']]
values_h_copertura_via = df_pred.loc[:,['h_copertura_via']]

values_progressiva = df_pred.loc[:,['progressiva']]

In [ ]:
point_pred = df0.loc[:,['long','lat']]
df0['h_perm'] = predict(points, values_h_perm, point_pred)
df0['h_quat'] = predict(points, values_h_quat, point_pred)
df0['IGE'] = predict(points, values_ige, point_pred)
df0['IGE_sec'] = predict(points, values_ige_sec, point_pred)
df0['IGE_SEC_PERC'] = predict(points, values_ige_sec_perc, point_pred)
df0['progressiva'] = predict(points, values_progressiva, point_pred)
df0['Litho'] = df_prop.lithotipo[df0.IGE].tolist()

df0['Tipo'] = predict(points, values_tipo, point_pred)
df0['Tipo_plus'] = predict(points, values_tipo_plus, point_pred)
df0['h_copertura'] = predict(points, values_h_copertura, point_pred)
df0['h_copertura_ril'] = predict(points, values_h_copertura_ril, point_pred)
df0['h_copertura_tri'] = predict(points, values_h_copertura_tri, point_pred)
df0['h_copertura_via'] = predict(points, values_h_copertura_via, point_pred)
df0['h_rilevato'] = predict(points, values_h_rilevato, point_pred)
df0['h_rilevato_true'] = predict(points, values_h_rilevato_true, point_pred)
df0['h_trincea'] = predict(points, values_h_trincea, point_pred)
df0['h_viadotto'] = predict(points, values_h_viadotto, point_pred)

df0['vol_dave'] = 4/3*np.pi*(df0.dave/2)**3 / 2
df0.fillna(0, inplace=True)
# attribuisco ad ogni karst il lithotipo principale
d1 = dict(zip(df_prop.index, df_prop.lithotipo))
df0['Litho'] = df0.replace({'IGE':d1})['IGE']

In [ ]:
with plt.style.context(("seaborn-pastel",)):
    fig, ax = plt.subplots(1,1, figsize=(22,4))
    ax.plot(df.POINT_X, df.progressiva)
    ax.scatter(df0.long, df0.progressiva, c='navy')
#     plt.xlim(df.POINT_X.min(), df[df.progressiva<40000]['POINT_X'].max())

In [ ]:
with plt.style.context(("seaborn-pastel",)):
    fig, ax = plt.subplots(1,1, figsize=(22,4))
    ax.plot(df.POINT_X, df.IGE)
    ax.scatter(df0.long, df0.IGE, c='navy')
    plt.xlim(df.POINT_X.min(), df[df.progressiva<40000]['POINT_X'].max())

In [ ]:
###  ATTENZIONE PER AVERE UNA BUONA CORRISPONDENZA KARST VS LITOGRAFIA PLOTTARE I KARST CHE SONO PROSSIMI AL TRACCIATO!!! 
#check 100 = True troppi pochi punti
# con check 500 si ha un buon compromesso tra distorsione data dalla distanza dall'asse e il numero di karst

In [ ]:
#with plt.style.context(("dark_background",)):
with plt.style.context(("seaborn-pastel",)):
    
    fig, ax = plt.subplots(1,1, figsize=(24,4))
    ax.plot(df.POINT_X, df.h_quat, label='quat thickness')
    ax.plot(df.POINT_X, df.h_perm, c='m', label='permian thickness')
    
    # se uso check  100 troppi pochi karst
    #ax.scatter(df0[df0.check_100 == True].long, df0[df0.check_100 == True].h_quat)
    #ax.scatter(df0.long, df0.h_quat)
    ax.scatter(df0[df0.check_500 == True].long, df0[df0.check_500 == True].h_quat)
    
    plt.legend()

    
    plt.xlim(df.POINT_X.min(), df[df.progressiva<40000]['POINT_X'].max())
    
    plt.savefig('Karst_andprofile.png')
    

In [ ]:
with plt.style.context(("seaborn-pastel",)):
    
    fig, ax = plt.subplots(1,1, figsize=(24,4))
    ax.plot(df.POINT_X, df.h_quat+df.h_perm, label='quat+perm thickness')
    ax.hlines(50,df.POINT_X.min(), df.POINT_X.max(), colors="m")
    
    #ax.scatter(df0.long, np.ones(df0.shape[0])*50, c="m", label="Karsts and subsidences")
    ax.scatter(df0[df0.check_500 == True].long, np.ones(df0[df0.check_500 == True].shape[0]), c="m", label="Karsts and subsidences")
    
    plt.legend()

    
    plt.xlim(df.POINT_X.min(), df[df.progressiva<40000]['POINT_X'].max())
    
    plt.savefig('Karst_andprofile_50mline.png')

In [ ]:
with plt.style.context(("dark_background",)):
    
    fig, ax = plt.subplots(1,1, figsize=(24,4))
    ax.plot(df.POINT_X, df_orig.m_s_l_m_, c="w", label="Ground Level")
    ax.plot(df.POINT_X, df_orig.calc_top, c="b", label="BedRock")
    ax.plot(df.POINT_X, df_orig.Arg_perm, c="c", label="Permian")

    #ax.scatter(df0.long, np.ones(df0.shape[0])*50, c="m", label="Karsts and subsidences")
    ax.scatter(df0[df0.check_500 == True].long, np.ones(df0[df0.check_500 == True].shape[0])*50, c="m", label="Karsts and subsidences")
    
    
    plt.legend()

    
    plt.xlim(df.POINT_X.min(), df[df.progressiva<40000]['POINT_X'].max())
    plt.savefig('Karst_andprofile_legend.png')


In [ ]:
sns.set(style="darkgrid")
 
# plotting both distibutions on the same figure
fig = sns.barplot(df0[df0.Litho=='s'].type, df0[df0.Litho=='s'].dave, color="r")
# fig = sns.barplot(df0[df0.Litho=='a'].type, df0[df0.Litho=='a'].dave, color="b")
plt.show()

### Calcolo Karst alla base con formula inversa

#### FUNZIONE PER IL CALCOLO INVERSO

In [ ]:
def calculate_dia(row, t, df_prop, df_prop_rock):
    
    # il diametro iniziale "d0" = 0.1 incrementato per step
    # when doing back calculation use t= 0
    
    lista_argille = df_prop[df_prop.lithotipo == 'a'].index.tolist()
    lista_sabbie = df_prop[df_prop.lithotipo == 's'].index.tolist()
    
    #==========================================================================================
    #                   QUI CREO LA LISTA CON GLI IGE IN QUEL KARST SPECIFICO
    #           PUO' ESSERE MESSA COME DATO DI INPUT SE INTERPOLATA ESTERNAMENTE
    #   OPPURE PUO ESSERE CREATA QUI DI SEGUITO PER INTERPOLAZIONE CON I BOREHOLES FITTIZI
    
    if row.h_perm>0:
        # TODO chiedere a Walter il permiano a che unità corrisponde
        # 14 --> J3 -->  "Clay grayish-brown, dusty, light, hard, J3
        list_layer = ['17']
        list_lathick = [row.h_perm]
    else:
        list_layer = []
        list_lathick  =[]

    if row.h_quat < 5:
        list_layer.append(row.IGE)
        list_lathick.append(row.h_quat)
    else:
        # se la litologia principale è argilla
        if df_prop.loc[row.IGE, 'lithotipo'] == 'a':
            # DO IT
            # crea una funzione che interpone un litotipo in sabbia
            # di spessore random uniform da 1/10 a 1/5 dello spessore totale dell'argilla
            list_layer.append(row.IGE)
            #list_layer.append(random.choice(lista_sabbie))
            list_layer.append(row.IGE_sec)
            list_layer.append(row.IGE)
            ige_perc = row.IGE_SEC_PERC
            randthick = random.uniform(row.h_quat*ige_perc*1.2, row.h_quat*ige_perc*0.8)
            list_lathick.append((row.h_quat - randthick)/2)
            list_lathick.append(randthick)
            list_lathick.append((row.h_quat - randthick)/2)
        else:  
            # se la litologia principale è sabbia
            # DO IT
            # crea una funzione che interpone un litotipo in argilla
            # di spessore random uniform da 1/10 a 1/5 dello spessore totale della sabbia  
            list_layer.append(row.IGE)
            list_layer.append(random.choice(lista_argille))
            list_layer.append(row.IGE)
            randthick = random.uniform(row.h_quat/10, row.h_quat/5)
            list_lathick.append((row.h_quat - randthick)/2)
            list_lathick.append(randthick)
            list_lathick.append((row.h_quat - randthick)/2)
    
    list_sv = []
    sv_temp = 0
    # invertire layer e calc sv
    for i in range(len(list_layer)):
        sv_temp += df_prop.gamma[i] * list_lathick[-1*(1+i)]
        list_sv.append(sv_temp)
    
    # come per list_layer ordino dal basso verso l'alto
    list_sv.reverse()
  
    #print(list_layer, list_lathick, df_prop.lithotipo[list_layer].tolist())
    
    # a questo punto ho la lista con litotipi e spessori
    # per le rocce usa solo calcari ma tieni in conto una futura revisione
    
    d_0 = 0.33
    d_target = row.dave
    sv = 0   # sigma inizializzata a zero
    
    # per la versione a ritroso d0 =0.1 e incrementato di 0.1 fin quando al top dello strato non si trova
    # lo stesso valore di dave

    
    #=================================================   inizio calcdia_strato   =================================================
    
    def calcdia_strato(df_prop, d_pod, layer_i, thick_i, sv, sv_i):
        # creare una funzione che restituisce il diametro strato superiore        
        # return i parametri che si aggiornano: d_top e sv 
        
        primoloop = True if sv == 0 else False
        
        radphi = np.radians(df_prop.phi[layer_i])
        _gamma = df_prop.gamma[layer_i]
        
        z1 = d_pod/2 * (1+2*np.sin(radphi))/(2*np.tan(radphi)*np.sin(radphi))
        
        x_2 = d_pod/2 * (1 + np.sin(radphi))**2/(4*np.sin(radphi)**3)
        X_2 = x_2 * 2
        
        z2 = z1 + d_pod/2*(1+np.sin(radphi))/(4*np.tan(radphi)*np.sin(radphi)**2)

        z3 = z2 + x_2/(np.tan(radphi))

        
        
        zone = 1 if(thick_i < z2) else (2 if thick_i < z3 else 3)   
        
        
        #sv = sv + df_prop.gamma[layer_i] * thick_i
        sv = 1 # per indicare loop successivi al primo
            
        eta = 0
       
        if df_prop.lithotipo[layer_i] == 'a':
            
            '''
            ********* ARGILLE  *******
            '''
                  
            _IL = float(df_prop.IL[layer_i])
            _c = df_prop.c[layer_i]
            if _c == 0: _c = 1
            _dfis = df_prop.Dfis[layer_i]
            

            dcrit = 2 * thick_i * ((np.tan(np.pi/4-radphi/2))**2 * np.tan(radphi) + 2 *_c * (1 - _IL)/sv_i)
            if dcrit < 0.01: dcrit = 0.01
            dpipe = np.exp(_IL) * _dfis * _gamma * thick_i / _c
            
            # parametro per il calcolo Pds
            eta = thick_i * dpipe / dcrit
            
            ## primo loop
            
            d_top = d_pod
            
            if (not primoloop) and (d_pod > dpipe): 
                if d_pod > dcrit:
                    d_top = dcrit
                else:
                    d_top = dpipe
            else:
                #d_top = d_pod   ********** ATTENZIONE ************
                d_top = d_pod
                pass
        
        else:
            '''
            *********  SABBIE   *******
            '''
            
            if zone == 1 :
                d_top = (X_2 - d_pod) / z2 * thick_i + d_pod
                #print('zone1 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)
            elif zone == 2 :
                d_top = X_2
                #print('zone2 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)
            else :
                #d_top = 0    ********** ATTENZIONE ************
                d_top = X_2
                #print('zone3 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)
        
        vol = np.pi * thick_i / 3 * (d_pod**2 / 4 + d_pod * d_top / 4 + d_top**2 / 4)
        
        if zone == 3: vol = vol + np.pi * d_pod**2 * (thick_i - z3) / 4
            
        e_i = float(df_prop.e[layer_i])
        ecr_i = float(df_prop["e,cr"][layer_i])
            
        deltav = (ecr_i - e_i) /(1 + ecr_i) * vol
        
        return d_top, sv, vol, eta
    
    #===========================================  end calcdia_strato      ============================================================  
    
    
    
    
    
    
    
    #================================================================================================================================        
        
    def calc_fixed_par(t, df_prop_rock, d_0, list_layer):
        
        # when doing back calculation use t=0
        
        v_r = df_prop_rock.v_r.Calcari
        h_p = d_0/2.5  # da controllare
        D1 = v_r * t + d_0
        m_k = df_prop_rock.m_k.Calcari
        k_k = df_prop_rock.k_k.Calcari
        phiradmedio = df_prop.phi[list_layer].mean()

        return(v_r, h_p, D1, m_k, k_k, phiradmedio)
    
    #===========================================   end  calc_fixed_par       =======================================================  
    
    
    v_r, h_p, d_0, m_k, k_k, phiradmedio  = calc_fixed_par(t, df_prop_rock, d_0, list_layer)  
    
    
    
    
    
    
    
    
    
    
    #===========================================   inizio ciclo iterativo       =======================================================
    step = 1
    report = 'other'
    
    while True:
        
        # inizializzo a d_0 finito il ciclo sarà quello in superficie
        d_top = d_0
           
        list_dia = [d_0] # il primo della lista è il d_pod
        list_vol = []
        list_eta = []
        
        for i, e in enumerate(list_layer):
            
            # calcola per tutti i layer quando esce dal loop d_0 = diam al piano campagna
            layer_i = e
            thick_i = list_lathick[i]
            sv_i = list_sv[i]
            
            d_top, sv, vol, eta = calcdia_strato(df_prop, d_top, layer_i, thick_i, sv, sv_i)
            
            list_dia.append(d_top)
            list_vol.append(vol)
            list_eta.append(eta)
            
        # calc subsidence
        
        dpod1 = list_dia[-2] # devo prendere la penultima perché l'ultima è in superficie
        sv1 = list_sv[-1]    # sv' al primo strato in alto (ultimo nella lista)
        lay1 = list_layer[-1] # layer del primo strato ovvero l'IGE principale
        th1  = list_lathick[-1] # thickness del primo strato superficiale (ultimo nella lista)
        
        d_sub = 2 * np.sqrt(th1**2 - (dpod1/2)**2)
        
        # per spessori piccoli th1 < raggio-pod quindi non c'è soluzione imporre un diametro minimo d_0 = 0.33
        # aumentando il dpod ad un certo punto ottengo Dpod/2 > th1 --> Nan
        # impondo il break
        # per entrare nel break impongo
        
        if (th1**2 - (dpod1/2)**2)<0: d_sub = d_target
        
            
        
        h_sub = sv1 * dpod1**3 * th1 * (1-df_prop.nu[lay1]**2) * (1+((df_prop.nu[lay1])/(1-df_prop.nu[lay1]))
                                                                 ) / (df_prop.E[lay1] * 1000 * ((4 * dpod1**2) - th1**2))
        # per evitare soluzioni negative
        if ((4 * dpod1**2) - th1**2) <= 0: h_sub = 100
               
        # d_0 è il d alla base stimato
        # d_top è quello calcolato a piano campagna (che appena supera il d_target stoppa i calcoli)
        
        #if (d_top < d_target) & (d_0 < d_target) & (row.type == 'Inbuto'):
        if (d_top < d_target) & (d_0 < d_target):
            step += 1
            d_0 += 0.33
            
            # ricomincia il loop da while con il valore d_0 incrementato
            if step >= 100: 
                d_0 = d_target
                #print('Risultato finale max iteration: ', d_0, d_top, list_dia)
                report = ('100_iter')
                break
            else:
                pass
        
        #elif (d_sub < d_target) & (d_0 < d_target) & (row.type != 'Inbuto'):
        elif (d_sub < d_target) & (d_0 < d_target):
            step += 1
            d_0 += 0.33
            
            # ricomincia il loop da while con il valore d_0 incrementato
            if step >= 100: 
                d_0 = d_target
                #print('Risultato finale max iteration: ', d_0, d_top, list_dia)
                report = ('100_iter')
                break
            else:
                pass
                
        else:
            # esci dal loop il primo valore d_0 che porta d_top > d_target è quello finale
            #print('Risultato finale : ', d_0, d_top, list_dia)
            report = ('sltn_found')
            break
    
    
    #===========================================   fine ciclo iterativo       =======================================================
    
    
    
    
    
    
    
    
    #===========================================    inizio calcprob         =========================================================    
    
    def calc_prob(df_prop, df_prop_rock, list_layer, d_top, v_r, h_p, D1, m_k, k_k, phiradmedio, t, list_vol, list_eta):

        # H_k = non richiesta
        
        Va_0 = np.pi * D1**2 * h_p/4
        
        Va_1 = np.pi * m_k * (k_k + v_r * t / m_k) * (3*D1**2 + ( 2 * m_k * (3*D1 + \
                2 * m_k / np.tan(phiradmedio)) / (np.tan(phiradmedio))))/12

        Va = Va_0 + Va_1
        
        list_Va0i = list_vol + Va_0
        
        sum_Va0i = np.array(list_Va0i).sum()
        sum_eta = np.array(list_eta).sum()
        sum_m = np.array(list_lathick).sum()

        Pds = 1 - np.exp(-Va / sum_Va0i * sum_eta / sum_m)

        return Pds
    
    #===========================================     fine  calcprob         ========================================================= 

    
    
    
    

    # Nel calcolo non a ritroso se Pds < 0.3 --> d_0 = 0
    
    #Pds = calc_prob(df_prop, df_prop_rock, list_layer, d_target, v_r, h_p, d_0, m_k, k_k, phiradmedio, t, list_vol, list_eta)
    
    if (row.h_perm == 0) & (row.h_quat <1) : d_0 = d_target
    
    #print("\n", "===========================================================================================================", "\n")
    
    return pd.Series([d_0, d_sub, h_sub, step, report])



In [ ]:
# nel df0 dbase è diametro al contatto con la roccia mentre nel df_zo è il diametro in superficie!

df0[['dbase', 'd_sub', 'h_sub', 'step', 'report']] = df0.apply(calculate_dia, axis=1, args=(0, df_prop, df_prop_rock))

In [ ]:
df0['vol_dbase'] = 4/3*np.pi*(df0.dbase/2)**3 / 2

## DA QUI INIZIANO LE 1000 SIMULAZIONI

- n_karst_soglia confermato = 70 (totale giovani + antichi)
- ritorno a considerare solo quelli giovani
- confermo : permiano > 40m d_top = 0
- confermo : permiano + quat > 50m d_top = 0
- la funzione calcola strato return d_top e Pds
- creo prima il df_zo poi seleziono il check100 per creare df_zo_100


### TROVA DISTRIBUZIONE PER OGNI ZONA E CREA STOCH

In [ ]:
def trova_distr_exp(df_g50, hh = 'none'):

    # cerco l'area con più karst
    # e quella con karst minimo (il minimo viene definito qui)
    # sotto un certo numero di karst per area uso una quantità minima prefissata
    n_karst_soglia = n_karst_soglia
    #size dei dati simulati
    size = size_exp
    # numero max in tutti i settori di 1km
    n_max_settore = int(df_g50.n_karst.max())
    # settore con numero max
    settore_nmax= df_g50[df_g50.n_karst == df_g50.n_karst.max()].sector2000_50.tolist()[0]
    # settore con numero karst basso ma maggiore della soglia (ex 40):
    settore_nmin= df_g50[df_g50.n_karst >= n_karst_soglia].sort_values("n_karst").sector2000_50.tolist()[0]
    
    # se non metto nulla negli attributi della def usa il max
    if hh  == 'none':
        h = settore_nmax
        
    elif df_g50[df_g50.sector2000_50==hh].n_karst.tolist()[0] < n_karst_soglia:
        h = settore_nmin
        
    else:
        h = hh
    
    # ricorda in df0 dbase e dave sono invertiti
    zo_dave = df0[(df0.sector2000_50==h)&(df0.era == "Giovane")].dbase
    # per aumentare la stima del numero considero anche quelli antichi
    #zo_dave = df0[df0.sector2000_50==h].dbase

    # TO DO TRY WITH st.truncexp
    #params = st.expon.fit(zo_dave, floc=0)
    #scale = params[1]/correzione_distr_exp
    
    
    #====================================================================================
    n_bins = int(zo_dave.max() - zo_dave.min())
    y, x = np.histogram(zo_dave.values, bins=n_bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0  #per ogni bin (x + x2)/2 trovo xc centroid
    x_inf = x[np.argmax(y)]
    x_sup = x.max()
    
    data_trunc = zo_dave[zo_dave>x_inf].values
    params = st.expon.fit(data_trunc, floc=0)
    scale = round(params[-1],3)
    
    # area sotto le barre da x_inf -> x.max()
    cdfdata = expon.cdf(x_sup, scale=scale) - expon.cdf(x_inf, scale=scale) 
    # area sotto le barre da 0 a x_inf  (quantità da stimare)
    cdfinf = expon.cdf(x_inf, scale=scale) 
    # area oltre x.max()
    cdfsup = 1 - expon.cdf(x_sup, scale=scale)

    k_range = int(zo_dave[zo_dave>x_inf].shape[0])
    k_inf = int(k_range/cdfdata*cdfinf)
    k_sup = int(k_range/cdfdata*cdfsup)
    
    k_tot_pop = k_range + k_inf + k_sup
    
    
    #===========================================================================================================
    

    zo_perc5 = int(np.percentile(zo_dave, 5))
    zo_perc95 = int(np.percentile(zo_dave, 95))
    zo_karstnum = zo_dave.shape[0]
    
    


    # calcolo le proporzioni tra i karst sotto il 5p
    # 1. zo_karstnum_simul è il numero su 1000 dei karst individuati a rilievo
    # 2. zo_nbelow5p_simul è la stima su 1000 dei karst non individuati
    # zo_tot è il totale della popolazione (tolto il frattile superiore al 95% che viene considerato un errore)
    # zo_tot_prop è la proporzione tra il totale della simulazione e 1+2
    # zo_pop_prop è la proporzione tra il totale della popolazione (compreso frattile >95%) e 1+2

    
    r = expon.rvs(scale=scale, size=size)

    zo_karstnum_simul = r[(zo_perc5<r)&(r<zo_perc95)].shape[0]
    zo_nbelow5p_simul = r[r<zo_perc5].shape[0]
    zo_tot = zo_karstnum_simul + zo_nbelow5p_simul
    zo_tot_prop = zo_tot/zo_karstnum_simul
    zo_pop_prop = size/zo_karstnum_simul

    zo_karstnum_simul, zo_nbelow5p_simul, zo_tot, zo_tot_prop, zo_pop_prop

    zo_nbelow5p = int(round(zo_karstnum/zo_karstnum_simul*zo_nbelow5p_simul,0))
    zo_nbelow5p

    # the number of data to obtain 27 karst less than 6m is
    # 37/538*1000

    zo_karstnum_pop = int(round(zo_karstnum/zo_karstnum_simul*r.shape[0],0))
    zo_karstnum_pop

    # zo_karstnum_pop = sono quelli totali della simulazione
    # zo_karstnum = sono quelli effettivi individuati 95p e non individuati)

    zo_karstnum_pop = int(round(zo_pop_prop *zo_karstnum, 0))
    
    # k_inf, k_tot_pop, k_range, k_sup
    # sono gli indici nuovi aggiunti con il nuovo approccio
    
    return zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale, k_inf, k_tot_pop, k_range, k_sup

### calcolo diametro per ogni strato

In [ ]:
def calculate_dia_diretto(row, t, df_prop, df_prop_rock, stochastic = False):
    
    '''
    input:
    row del df_zo
    t = int(vita utile dell'opera serve per calcolare la dissoluzione e il dia finale)
    df_prop = dataframe con le proprietà per ogni IG
    df_prop_rock = dataframe con le proprietpà del substrato
    stochastic = False use deterministic values True use uniform distribution MIN-MAX
    return:
    d_top : float (diametro parte alta dello strato)
    
    versioni:
    1° deterministico, ogni d_0 crea un d_top, e Pds se permiano e quat = 0 d_top = d_0

    2° prob usa MIN-MAX, d_top = 0 se h_perm > 40 o se h_perm+h_quat >50
    
    3°  considerazioni circa il Pds e la formazione di inbuti o subsidenze
        si fanno a posteriori
        
    4° Nota assunzione df['h_perm'] = df_orig['h_perm'] + df_orig['h_giur'] + df_orig['h_marne'] --> unità 17
        scorporare il h_perm --> unità 17
                      h_giur --> unità 16
                      h_marne --> unità 14
    '''
    
    d_0 = row.dave
    
    lista_argille = df_prop[df_prop.lithotipo == 'a'].index.tolist()
    lista_sabbie = df_prop[df_prop.lithotipo == 's'].index.tolist()
    
    
    #==========================================================================================
    #                   QUI CREO LA LISTA CON GLI IGE IN QUEL KARST SPECIFICO
    #           PUO' ESSERE MESSA COME DATO DI INPUT SE INTERPOLATA ESTERNAMENTE
    #   OPPURE PUO ESSERE CREATA QUI DI SEGUITO PER INTERPOLAZIONE CON I BOREHOLES FITTIZI
    
    
    if row.h_perm>0:
        list_layer = ['17']
        list_lathick = [row.h_perm]
    else:
        list_layer = []
        list_lathick  =[]

    if row.h_quat < 5:
        list_layer.append(row.IGE)
        list_lathick.append(row.h_quat)
    else:
        # se la litologia principale è argilla
        if df_prop.loc[row.IGE, 'lithotipo'] == 'a':
            # DO IT
            # crea una funzione che interpone un litotipo in sabbia
            # di spessore random uniform da 1/10 a 1/5 dello spessore totale dell'argilla
            list_layer.append(row.IGE)
            #list_layer.append(random.choice(lista_sabbie))
            list_layer.append(row.IGE_sec)
            list_layer.append(row.IGE)
            ige_perc = row.IGE_SEC_PERC
            randthick = random.uniform(row.h_quat*ige_perc*1.2, row.h_quat*ige_perc*0.8)
            list_lathick.append((row.h_quat - randthick)/2)
            list_lathick.append(randthick)
            list_lathick.append((row.h_quat - randthick)/2)
        else:  
            # se la litologia principale è sabbia
            # DO IT
            # crea una funzione che interpone un litotipo in argilla
            # di spessore random uniform da 1/10 a 1/5 dello spessore totale della sabbia  
            list_layer.append(row.IGE)
            list_layer.append(random.choice(lista_argille))
            list_layer.append(row.IGE)
            randthick = random.uniform(row.h_quat/10, row.h_quat/5)
            list_lathick.append((row.h_quat - randthick)/2)
            list_lathick.append(randthick)
            list_lathick.append((row.h_quat - randthick)/2)
    
    list_sv = []
    sv_temp = 0
    # invertire layer e calc sv
    for i in range(len(list_layer)):
        sv_temp += df_prop.gamma[i] * list_lathick[-1*(1+i)]
        list_sv.append(sv_temp)
    
    # come per list_layer ordino dal basso verso l'alto
    list_sv.reverse()

    # a questo punto ho la lista con litotipi e spessori
    # per le rocce usa solo calcari ma tieni in conto una futura revisione
    
    #===================================================================================================

    nloop = 0   # nloop inizializzato a zero

    #=================================================   inizio calcdia_strato   =================================================

    def calcdia_strato(df_prop, d_pod, layer_i, thick_i, nloop, sv_i):
        # creare una funzione che restituisce il diametro strato superiore        
        # return i parametri che si aggiornano: d_top e sv 

        primoloop = True if nloop == 0 else False

        _phi = df_prop.phi[layer_i]
        if stochastic: _phi = random.uniform(df_prop.phi_MIN[layer_i], df_prop.phi_MAX[layer_i])
        # nel caso in cui min max non siano definiti
        if _phi == 0: _phi = df_prop.phi[layer_i]
        radphi = np.radians(_phi)

        _gamma = df_prop.gamma[layer_i]
        if stochastic: _gamma = random.uniform(df_prop.gamma_MIN[layer_i], df_prop.gamma_MAX[layer_i])
        # nel caso in cui min max non siano definiti
        if _gamma == 0: _gamma = df_prop.gamma[layer_i]
            
        z1 = d_pod/2 * (1+2*np.sin(radphi))/(2*np.tan(radphi)*np.sin(radphi))
        x_2 = d_pod/2 * (1 + np.sin(radphi))**2/(4*np.sin(radphi)**3)
        X_2 = x_2 * 2
        z2 = z1 + d_pod/2*(1+np.sin(radphi))/(4*np.tan(radphi)*np.sin(radphi)**2)
        z3 = z2 + x_2/(np.tan(radphi))

        zone = 1 if(thick_i < z2) else (2 if thick_i < z3 else 3)   

        #sv = sv + df_prop.gamma[layer_i] * thick_i
        nloop += 1 # per indicare loop successivi al primo

        eta = 0

        if df_prop.lithotipo[layer_i] == 'a':

            '''
            ********* ARGILLE  *******
            '''

            _IL = float(df_prop.IL[layer_i])
            # ci sono casi in cui il IL risulta negativo
            if _IL <0 : _IL = 0.05 


            _c = df_prop.c[layer_i]              
            if stochastic: _c = random.uniform(df_prop.c_MIN[layer_i], df_prop.c_MAX[layer_i])
            # nel caso in cui min max non siano definiti
            if _c == 0: df_prop.c[layer_i]
            # se c è effettivamente prossimo a zero metter un valore minimo
            if _c == 0: _c = 1


            _dfis = df_prop.Dfis[layer_i]
            if stochastic: _dfis = random.uniform(df_prop.Dfis_MIN[layer_i], df_prop.Dfis_MAX[layer_i])
            # nel caso in cui min max non siano definiti
            if _dfis == 0: df_prop.Dfis[layer_i]


            dcrit = 2 * thick_i * ((np.tan(np.pi/4-radphi/2))**2 * np.tan(radphi) + 2 *_c * (1 - _IL)/sv_i)
            if dcrit < 0.01: dcrit = 0.01
            dpipe = np.exp(_IL) * _dfis * _gamma * thick_i / _c

            # parametro per il calcolo Pds
            eta = thick_i * dpipe / dcrit

            ## primo loop

            d_top = d_pod

            if (not primoloop) and (d_pod > dpipe): 
                if d_pod > dcrit:
                    d_top = dcrit
                else:
                    d_top = dpipe
            else:
                #per il primo loop solamente o se d_pod < dpipe
                #d_top = d_pod   ********** ATTENZIONE ************
                d_top = d_pod
                pass

        else:
            '''
            *********  SABBIE   *******
            '''

            if zone == 1 :
                d_top = (X_2 - d_pod) / z2 * thick_i + d_pod
                ##print('zone1 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)
            elif zone == 2 :
                d_top = X_2
                ##print('zone2 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)
#                 elif thick_i > (z3*3):
#                     #print("thick_i > (z3*3)")
#                     d_top = 0
            else :
                #d_top = 0    ********** ATTENZIONE ************
                d_top = X_2
                ##print('zone3 :',zone,'z2',z2,'z3',z3,'dpod: ',d_pod, 'phi: ', df_prop.phi[layer_i], 'radphi: ', radphi, 'gamma: ', _gamma)

        vol = np.pi * thick_i / 3 * (d_pod**2 / 4 + d_pod * d_top / 4 + d_top**2 / 4)

        if zone == 3: vol = vol + np.pi * d_pod**2 * (thick_i - z3) / 4

        e_i = float(df_prop.e[layer_i])
        ecr_i = float(df_prop["e,cr"][layer_i])

        deltav = (ecr_i - e_i) /(1 + ecr_i) * vol

        return d_top, nloop, vol, eta

    #===========================================  end calcdia_strato      ============================================================  







    #================================================================================================================================        

    def calc_fixed_par(t, df_prop_rock, d_0, list_layer):

        # when doing back calculation use t=0

        v_r = df_prop_rock.v_r.Calcari
        h_p = d_0/2.5  # da controllare
        D1 = v_r * t + d_0
        m_k = df_prop_rock.m_k.Calcari
        k_k = df_prop_rock.k_k.Calcari
        phiradmedio = df_prop.phi[list_layer].mean()

        return(v_r, h_p, D1, m_k, k_k, phiradmedio)

    #===========================================   end  calc_fixed_par       =======================================================  


    # sovrascrivo d_0 a quello originale in teoria dovrebbe chiamarsi D1 ovverso il diametro d_0 allargato con dissoluzione a t anni

    v_r, h_p, d_0, m_k, k_k, phiradmedio  = calc_fixed_par(t, df_prop_rock, d_0, list_layer)  










    #===========================================   inizio calcolo       =======================================================


    d_top = d_0

    list_dia = [d_0] # il primo della lista è il d_pod
    list_vol = []
    list_eta = []

    for i, e in enumerate(list_layer):
        # calcola per tutti i layer quando esce dal loop d_0 = diam al piano campagna
        layer_i = e
        thick_i = list_lathick[i]
        sv_i = list_sv[i]
        
        d_top, nloop, vol, eta = calcdia_strato(df_prop, d_top, layer_i, thick_i, nloop, sv_i)

        list_dia.append(d_top)
        list_vol.append(vol)
        list_eta.append(eta)

    # d_0 è il d alla base (INPUT)
    # d_top è quello calcolato a piano campagna (OUTPUT)


    #===========================================   fine ciclo iterativo       =======================================================



    #===========================================    inizio calc subsidenze    =======================================================
    # TODO: aggiungere un check if list_dia[-2] == 0 allora prendere il precedente [-3] ecc...    
    
    
    dpod1 = list_dia[-2] # _pod che genera subsidenza devo prendere la penultima perché l'ultima è in superficie
    sv1 = list_sv[-1]    # sv' al primo strato in alto (ultimo nella lista)
    lay1 = list_layer[-1] # layer del primo strato ovvero l'IGE principale
    th1  = list_lathick[-1] # thickness del primo strato superficiale (ultimo nella lista)
    
    d_sub = 2 * np.sqrt(th1**2 - (dpod1/2)**2)
    # per spessori di strato modesti gestisco radici negative 
    if (th1**2 - (dpod1/2)**2)<0: d_sub = dpod1
           
    h_sub = sv1 * dpod1**3 * th1 * (1-df_prop.nu[lay1]**2) * (1+((df_prop.nu[lay1])/(1-df_prop.nu[lay1]))
                                                             ) / (df_prop.E[lay1] * 1000 * ((4 * dpod1**2) - th1**2))
    # per evitare soluzioni negative 
    # associare alla subsidenza un numero alto, si associerà anziche una subsidenza un inbuto
    if ((4 * dpod1**2) - th1**2) <= 0: h_sub = 100
    
    
        
    #===========================================    inizio calcprob         =========================================================    

    def calc_prob(df_prop, df_prop_rock, list_layer, d_top, v_r, h_p, D1, m_k, k_k, phiradmedio, t, list_vol, list_eta):

        # H_k = non richiesta

        Va_0 = np.pi * D1**2 * h_p/4

        Va_1 = np.pi * m_k * (k_k + v_r * t / m_k) * (3*D1**2 + ( 2 * m_k * (3*D1 + \
                2 * m_k / np.tan(phiradmedio)) / (np.tan(phiradmedio))))/12

        Va = Va_0 + Va_1

        list_Va0i = list_vol + Va_0

        sum_Va0i = np.array(list_Va0i).sum()
        sum_eta = np.array(list_eta).sum()
        sum_m = np.array(list_lathick).sum()

        Pds = 1 - np.exp(-Va / sum_Va0i * sum_eta / sum_m)

        return Pds

    #===========================================     fine  calcprob         ========================================================= 





    # Aggiungere che se la Pds < 0.3 si forma una sinclinale con diametro da calcolare

    Pds = calc_prob(df_prop, df_prop_rock, list_layer, d_top, v_r, h_p, d_0, m_k, k_k, phiradmedio, t, list_vol, list_eta)
    #print("Pds is :", Pds, "d_top is: ", d_top)

    #  IL CALCOLO IN FUNZIONE DI PDS LO FACCIO MANUALMENTE DOPO
    #         if ((Pds < 0.1) & (row.h_perm <20)): 
    #             d_top = 0
    #             #print('(Pds < 0.3) & (row.h_perm <20)')
    #         elif ((Pds < 0.3) & (row.h_perm >=20)):
    #             d_top = 0
    #             #print('(Pds < 0.5) & (row.h_perm >20)')
    #         else:
    #             pass
            

    
    ##print("\n", "===========================================================================================================", "\n")

    return pd.Series([d_top, Pds, d_sub, h_sub])


### CREO UN QUADRATO GRANDE ATTORNO ALL'AREA

In [ ]:
def create_df_zo(df_g50, h, zo_nbelow5p, zo_karstnum, zo_karstnum_pop, df_pred, scale):
    # per i diametri
    # devo fare una distribuzione con 68 elementi e inserire solo i primi 
    # 27 elementi ordinati per grandezza

    xmin = min(df_g50.loc[df_g50.index[h],'vtrackdown2000x'],
                               df_g50.loc[df_g50.index[h+1],'vtrackdown2000x'],
                               df_g50.loc[df_g50.index[h],'vtrackup2000x'],
                               df_g50.loc[df_g50.index[h+1],'vtrackup2000x']
                              )
    xmax = max(df_g50.loc[df_g50.index[h],'vtrackdown2000x'],
                               df_g50.loc[df_g50.index[h+1],'vtrackdown2000x'],
                               df_g50.loc[df_g50.index[h],'vtrackup2000x'],
                               df_g50.loc[df_g50.index[h+1],'vtrackup2000x']
                              )
    ymin = min(df_g50.loc[df_g50.index[h],'vtrackdown2000y'],
                               df_g50.loc[df_g50.index[h+1],'vtrackdown2000y'],
                               df_g50.loc[df_g50.index[h],'vtrackup2000y'],
                               df_g50.loc[df_g50.index[h+1],'vtrackup2000y']
                              )
    ymax = max(df_g50.loc[df_g50.index[h],'vtrackdown2000y'],
                               df_g50.loc[df_g50.index[h+1],'vtrackdown2000y'],
                               df_g50.loc[df_g50.index[h],'vtrackup2000y'],
                               df_g50.loc[df_g50.index[h+1],'vtrackup2000y']
                              )
    area_gr = (xmax-xmin)*(ymax-ymin)/1e6
    n_karst_5p_areag = int(round(zo_nbelow5p/df_g50.sector_area[h]*area_gr,0))
    # 5p + visiblekarst
    n_karst_areag = int(round(zo_nbelow5p + zo_karstnum / df_g50.sector_area[h] * area_gr,0))
    # 5p + visiblekarst + outofsize
    zo_karstnum_pop_areag = int(round(zo_karstnum_pop / df_g50.sector_area[h] * area_gr,0))

    # n_karst_5p_areag, n_karst_areag, zo_karstnum_pop_areag

    stoch_karstx = np.random.uniform(xmin, xmax, n_karst_areag)
    stoch_karsty = np.random.uniform(ymin, ymax, n_karst_areag)
    # creo the full curve in the areagrande areag estraggo i primi n-karst-5p-areag valori
    stoch_karstdia = expon.rvs(scale=scale, size=zo_karstnum_pop_areag)
    stoch_karstdia.sort()
    stoch_karstdia = stoch_karstdia[:n_karst_areag]

    #### creo l'array con tutti i karst sotchastici nell'area grande

    zo_arr = np.vstack([stoch_karstx, stoch_karsty, stoch_karstdia]).T

    #### Rimuovo i karst fuori dall'area di settore

    def abs_sum(row, i):
        check = False
        polygon = Polygon([tuple(df_g50[['vtrackup2000x','vtrackup2000y']].loc[i,:]),
                      tuple(df_g50[['vtrackup2000x','vtrackup2000y']].loc[i+1,:]),
                      tuple(df_g50[['vtrackdown2000x','vtrackdown2000y']].loc[i+1,:]),
                      tuple(df_g50[['vtrackdown2000x','vtrackdown2000y']].loc[i,:])])

        point = Point(row[0], row[1])

        if polygon.contains(point): check = True

        return check

    zo_arr = np.vstack([zo_arr.T, np.apply_along_axis(abs_sum, 1, zo_arr, h)]).T
    zo_arr = zo_arr[zo_arr[:,3]==True][:,:3]
    df_zo = pd.DataFrame(zo_arr, columns=['long','lat','dave'])
    # rinomino le colonne 
    # zo_arr = np.vstack([stoch_karstx, stoch_karsty, stoch_karstdia])
    # "dave" qui è il d alla base attenzione ho invertito nel df0 dave è al piano campagna

    df_zo['sector2000_50'] = int(h)           # per associazione con df0 e df_g50
    df_zo['sector2000_50_str'] = str(h)     # per associazione con df0
    
    points = df_pred.loc[:,['POINT_X','POINT_Y']]
    values_h_perm = df_pred.loc[:,['h_perm']]
    values_h_quat = df_pred.loc[:,['h_quat']]
    values_ige = df_pred.loc[:,['IGE']]
    values_ige_sec = df_pred.loc[:,['IGE_sec']]
    values_ige_sec_perc = df_pred.loc[:,['IGE_SEC_PERC']]
    values_tipo = df_pred.loc[:,['Tipo']]
    values_h_rilevato = df_pred.loc[:,['h_rilevato']]

    point_pred = df_zo.loc[:,['long','lat']]
    df_zo['h_perm'] = predict(points, values_h_perm, point_pred)
    df_zo['h_quat'] = predict(points, values_h_quat, point_pred)
    df_zo['IGE'] = predict(points, values_ige, point_pred)
    df_zo['IGE_sec'] = predict(points, values_ige_sec, point_pred)
    df_zo['IGE_SEC_PERC'] = predict(points, values_ige_sec_perc, point_pred)
    df_zo['Tipo'] = predict(points, values_tipo, point_pred)
    df_zo['h_rilevato'] = predict(points, values_h_rilevato, point_pred)
    


    #### Calcolo diametro in superficie
    
    df_zo[['dbase','Pds', 'd_sub', 'h_sub']] = df_zo.apply(calculate_dia_diretto, axis=1, args=(100, df_prop, df_prop_rock, True))
    df_zo['dbase'] = df_zo['dbase'].astype('float', errors='ignore')
    
    def create_df_zo_100(row, df_g50, df_zo):

        for i in range(df_g50.shape[0]-1):

            check = False

            polygon = Polygon([tuple(df_g50[['vtrackupx','vtrackupy']].loc[i,:]),
                          tuple(df_g50[['vtrackupx','vtrackupy']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdownx','vtrackdowny']].loc[i+1,:]),
                          tuple(df_g50[['vtrackdownx','vtrackdowny']].loc[i,:])])

            point = Point(row['long'], row['lat'])

            if polygon.contains(point): 
                check = True        
                break
            else:
                pass

        return check
    
    df_zo['check_100'] = df_zo.apply(create_df_zo_100, axis=1, args = (df_g50, df_zo))
    
    return df_zo



#### Applico la def "create_df_zo" per creare il dataframe con la colonna dbase --> il diametro in superficie  --> una sola iterazione stochastica
#### Risultati non utizzati

In [ ]:
df_zo = pd.DataFrame(columns=['long', 'lat', 'dave', 'sector2000_50', 'sector2000_50_str', 'h_perm',
       'h_quat', 'IGE', 'IGE_sec', 'IGE_SEC_PERC', 'dbase','Tipo','h_rilevato','check_100'])

#zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale = trova_distr_exp(df_g50)


##109
## pkfinale = 40
for hh in range (int(1000 / (div_settori * m_punti) * pkfinale)):
    
    zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale, k_inf, k_tot_pop, k_range, k_sup = trova_distr_exp(df_g50, hh==hh)
    df_zo = df_zo.append(create_df_zo(df_g50, hh, zo_nbelow5p, zo_karstnum, zo_karstnum_pop, df_pred, scale), ignore_index=True, sort=True)
    
# associo le tipologie di dato corrette

df_zo['dbase'] = df_zo['dbase'].astype('float')
df_zo['sector2000_50'] = df_zo['sector2000_50'].astype('int')

In [ ]:
df_zo[df_zo.dbase>0]

## Creo file df_zo  con iterazioni stochastiche per mappe
## Creo file df_zo_100 ma con iterazioni stochastiche e solo nell'intorno 100m

####  Funzioni propedeutiche al calcolo

In [ ]:
def rilevato_ksmall(row):
    
    if (row['Tipo'] == 'Rilevato') & (row['dbase'] < soglia_dia_karst):
        quantity = 1     
    else:
        quantity = 0
    return quantity


def rilevato_klarge(row):
    
    if (row['Tipo'] == 'Rilevato') & (row['dbase'] >= soglia_dia_karst): 
        quantity = 1
    else:
        quantity = 0    
    return quantity

def trincea_ksmall(row):
    
    if (row['Tipo'] == 'Trincea') & (row['dbase'] < soglia_dia_karst):
        quantity = 1     
    else:
        quantity = 0
    return quantity


def trincea_klarge(row):
    
    if (row['Tipo'] == 'Trincea') & (row['dbase'] >= soglia_dia_karst): 
        quantity = 1
    else:
        quantity = 0    
    return quantity


def vidotto_ksmall(row):
    
    if (row['Tipo'] != 'Rilevato') & (row['Tipo'] != 'Trincea') & (row['dbase'] < soglia_dia_karst) & (row['h_rilevato'] > lim_via_bridge):        
        quantity = 1        
    else:        
        quantity = 0        
    return quantity


def vidotto_klarge(row):
    
    if (row['Tipo'] != 'Rilevato') & (row['Tipo'] != 'Trincea') & (row['dbase'] >= soglia_dia_karst) & (row['h_rilevato'] > lim_via_bridge):        
        quantity = 1        
    else:        
        quantity = 0        
    return quantity


def passaggio_ksmall(row):
    
    if (row['Tipo'] != 'Rilevato') & (row['Tipo'] != 'Trincea') & (row['dbase'] < soglia_dia_karst) & (row['h_rilevato'] <= lim_via_bridge):        
        quantity = 1        
    else:        
        quantity = 0        
    return quantity


def passaggio_klarge(row):
    
    if (row['Tipo'] != 'Rilevato') & (row['Tipo'] != 'Trincea') & (row['dbase'] >= soglia_dia_karst) & (row['h_rilevato'] <= lim_via_bridge):        
        quantity = 1        
    else:        
        quantity = 0        
    return quantity

#### INIZIO IL CALCOLO STOCHASTICO

In [ ]:
# inizializzo df_zo

df_zo = pd.DataFrame(columns=['long', 'lat', 'dave', 'sector2000_50', 'sector2000_50_str', 'h_perm',
       'h_quat', 'IGE', 'IGE_sec', 'IGE_SEC_PERC', 'dbase', 'Tipo', 'h_rilevato', 'Pds', 'check_100', 'epoch'])

# se voglio una simulazione con la distribuzione spaziale più sfigata
#zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale = trova_distr_exp(df_g50, hh='none')

#======================================================================================================

for epoch in trange(epochs, file=sys.stdout, desc='epoch advance'):
    
    for hh in range (int(1000 / (div_settori * m_punti) * pkfinale)):
        
        zo_nbelow5p, zo_karstnum_pop, zo_karstnum, settore_nmax, scale, k_inf, k_tot_pop, k_range, k_sup = trova_distr_exp(df_g50, hh=hh)
        
        row_temp = create_df_zo(df_g50, hh, zo_nbelow5p, zo_karstnum, zo_karstnum_pop, df_pred, scale)
        row_temp['epoch'] = epoch
        
        # sort=True --> to avoid warning appearing
        df_zo = df_zo.append(row_temp, ignore_index=True, sort=False)
        
#======================================================================================================
    
# nel df_zo_100 devono rimanere i punti che stanno nella fascia 100m e che hanno dia > 0
df_zo_100 = df_zo[df_zo.check_100 == True]
df_zo_100 = df_zo_100[df_zo_100.dbase > 0]

# creo colonne per la definizione delle quantita di interventi
try:
    df_zo_100['ril_ksmall'] = df_zo_100.apply(rilevato_ksmall, axis=1)
    df_zo_100['ril_klarge'] = df_zo_100.apply(rilevato_klarge, axis=1)
    df_zo_100['tri_ksmall'] = df_zo_100.apply(trincea_ksmall, axis=1)
    df_zo_100['tri_klarge'] = df_zo_100.apply(trincea_klarge, axis=1)
    df_zo_100['via_ksmall'] = df_zo_100.apply(vidotto_ksmall, axis=1)
    df_zo_100['via_klarge'] = df_zo_100.apply(vidotto_klarge, axis=1)
    df_zo_100['pas_ksmall'] = df_zo_100.apply(passaggio_ksmall, axis=1)
    df_zo_100['pas_klarge'] = df_zo_100.apply(passaggio_klarge, axis=1)
except:
    pass

# area della semisfera

df_zo['vol_dave'] = 4/3*np.pi*(df_zo.dave/2)**3 / 2
df_zo_100['vol_dave'] = 4/3*np.pi*(df_zo_100.dave/2)**3 / 2
df_zo['Litho'] = df_prop.lithotipo[df_zo.IGE].tolist()

df_zo.fillna(0, inplace=True)
df_zo_100.fillna(0, inplace=True)

df_zo['h_copertura'] = df_zo.h_quat + df_zo.h_perm
df_zo_100['h_copertura'] = df_zo.h_quat + df_zo.h_perm
df_zo['dia_subimb']=df_zo.loc[:,['d_sub','dbase']].max(axis=1)
    
df_zo.to_csv('df_zo.csv', sep=';')
df_zo_100.to_csv('df_zo_100.csv', sep=';')
df.to_csv('df.csv', sep=';')
df0.to_csv('df0.csv', sep=';')
df.to_csv('df.csv', sep=';')
df_g50.to_csv('df_g50.csv', sep=';')


## ===================================================================================

#### MAKE df0 similar to df_zo mancano check_100

In [ ]:
# ###  gia fatto!!!  ##########
# point_pred = df0.loc[:,['long','lat']]
# df0['h_perm'] = predict(points, values_h_perm, point_pred)
# df0['h_quat'] = predict(points, values_h_quat, point_pred)
# df0['IGE'] = predict(points, values_ige, point_pred)
# df0['Tipo'] = predict(points, values_tipo, point_pred)
# df0['h_rilevato'] = predict(points, values_h_rilevato, point_pred)

In [ ]:
df0_100 = df0[df0.check_100==True]

In [ ]:
try:
    df0_100['ril_ksmall'] = df0_100.apply(rilevato_ksmall, axis=1)
    df0_100['ril_klarge'] = df0_100.apply(rilevato_klarge, axis=1)
    df0_100['tri_ksmall'] = df0_100.apply(trincea_ksmall, axis=1)
    df0_100['tri_klarge'] = df0_100.apply(trincea_klarge, axis=1)
    df0_100['via_ksmall'] = df0_100.apply(vidotto_ksmall, axis=1)
    df0_100['via_klarge'] = df0_100.apply(vidotto_klarge, axis=1)
    df0_100['pas_ksmall'] = df0_100.apply(passaggio_ksmall, axis=1)
    df0_100['pas_klarge'] = df0_100.apply(passaggio_klarge, axis=1)
except:
    pass

In [ ]:
df0_100['epoch'] = 100000
df0_100['h_copertura'] = df0_100.h_quat + df0_100.h_perm
df0_100['Pds'] = np.where(df0_100.type == 'Subsidenza', 0.09, 1)
df0_100 = df0_100[df_zo_100.columns.tolist()]
df0_100.fillna(0, inplace=True)

#### Completa df_g50 con i risultati dell'analisi

In [ ]:
df_zo['stipo'] = np.where(((df_zo.Pds < 0.1)&(df_zo.Pds>=0.01)&(df_zo.h_sub <=2)), 'subsidenza', 'imbuto')
df_zo['stipo'] = np.where(((df_zo.Pds < 0.01)|(df_zo.dbase == 0)),'none',df_zo.stipo)
df_g50['s_dsub_min'] = df_zo[df_zo.stipo == 'subsidenza'][['d_sub','sector2000_50']].groupby('sector2000_50').min()
df_g50['s_dsub_mean'] = df_zo[df_zo.stipo == 'subsidenza'][['d_sub','sector2000_50']].groupby('sector2000_50').mean()
df_g50['s_dsub_q99'] = df_zo[df_zo.stipo == 'subsidenza'][['d_sub','sector2000_50']].groupby('sector2000_50').quantile(.99)
df_g50['s_dsub_max'] = df_zo[df_zo.stipo == 'subsidenza'][['d_sub','sector2000_50']].groupby('sector2000_50').max()
df_g50['s_dsub_count'] = df_zo[df_zo.stipo == 'subsidenza'][['d_sub','sector2000_50']].groupby('sector2000_50').count()
df_g50['s_dbase_min'] = df_zo[df_zo.stipo == 'imbuto'][['dbase','sector2000_50']].groupby('sector2000_50').min()
df_g50['s_dbase_mean'] = df_zo[df_zo.stipo == 'imbuto'][['dbase','sector2000_50']].groupby('sector2000_50').mean()
df_g50['s_dbase_q99'] = df_zo[df_zo.stipo == 'imbuto'][['dbase','sector2000_50']].groupby('sector2000_50').quantile(.99)
df_g50['s_dbase_max'] = df_zo[df_zo.stipo == 'imbuto'][['dbase','sector2000_50']].groupby('sector2000_50').max()
df_g50['s_dbase_count'] = df_zo[df_zo.stipo == 'imbuto'][['dbase','sector2000_50']].groupby('sector2000_50').count()
df_g50['s_dsub_min_100'] = df_zo[(df_zo.stipo == 'subsidenza')&(df_zo.check_100 == True)][['d_sub','sector2000_50']].groupby('sector2000_50').min()
df_g50['s_dsub_mean_100'] = df_zo[(df_zo.stipo == 'subsidenza')&(df_zo.check_100 == True)][['d_sub','sector2000_50']].groupby('sector2000_50').mean()
df_g50['s_dsub_q99_100'] = df_zo[(df_zo.stipo == 'subsidenza')&(df_zo.check_100 == True)][['d_sub','sector2000_50']].groupby('sector2000_50').quantile(.99)
df_g50['s_dsub_max_100'] = df_zo[(df_zo.stipo == 'subsidenza')&(df_zo.check_100 == True)][['d_sub','sector2000_50']].groupby('sector2000_50').max()
df_g50['s_dsub_count_100'] = df_zo[(df_zo.stipo == 'subsidenza')&(df_zo.check_100 == True)][['d_sub','sector2000_50']].groupby('sector2000_50').count()
df_g50['s_dbase_min_100'] = df_zo[(df_zo.stipo == 'imbuto')&(df_zo.check_100 == True)][['dbase','sector2000_50']].groupby('sector2000_50').min()
df_g50['s_dbase_mean_100'] = df_zo[(df_zo.stipo == 'imbuto')&(df_zo.check_100 == True)][['dbase','sector2000_50']].groupby('sector2000_50').mean()
df_g50['s_dbase_q99_100'] = df_zo[(df_zo.stipo == 'imbuto')&(df_zo.check_100 == True)][['dbase','sector2000_50']].groupby('sector2000_50').quantile(.99)
df_g50['s_dbase_max_100'] = df_zo[(df_zo.stipo == 'imbuto')&(df_zo.check_100 == True)][['dbase','sector2000_50']].groupby('sector2000_50').max()
df_g50['s_dbase_count_100'] = df_zo[(df_zo.stipo == 'imbuto')&(df_zo.check_100 == True)][['dbase','sector2000_50']].groupby('sector2000_50').count()

#### Tabella con diametri min-max-ave e quantili

In [ ]:
df_g50[['sector2000_50',
        's_dsub_min','s_dsub_mean','s_dsub_q99','s_dsub_max','s_dsub_count',
        's_dbase_min','s_dbase_mean','s_dbase_q99','s_dbase_max','s_dbase_count',
        's_dsub_min_100','s_dsub_mean_100','s_dsub_q99_100','s_dsub_max_100','s_dsub_count_100',
        's_dbase_min_100','s_dbase_mean_100','s_dbase_q99_100','s_dbase_max_100', 's_dbase_count_100']
      ][df_g50.sector2000_50<=39].to_csv('diametriminmaxave.csv', sep=';')

In [ ]:
df_zo.to_csv('df_zo.csv', sep=';')
df_zo_100.to_csv('df_zo_100.csv', sep=';')
df.to_csv('df.csv', sep=';')
df0.to_csv('df0.csv', sep=';')
df.to_csv('df.csv', sep=';')
df_g50.to_csv('df_g50.csv', sep=';')
df0_100.to_csv('df0_100.csv', sep=';')

# =====================================================

### Per farlo partire senza far girare la macro caricare i file di salvataggio

In [ ]:
df_zo = pd.read_csv('df_zo.csv', sep=';', index_col=0)
df_zo_100 = pd.read_csv('df_zo_100.csv', sep=';', index_col=0)
df = pd.read_csv('df.csv', sep=';', index_col=0)
df0 = pd.read_csv('df0.csv', sep=';', index_col=0)
df0_100 = pd.read_csv('df0_100.csv', sep=';', index_col=0)
df = pd.read_csv('df.csv', sep=';', index_col=0)
df_g50 = pd.read_csv('df_g50.csv', sep=';', index_col=0)
df0_100 = pd.read_csv('df0_100.csv', sep=';', index_col=0)

## Plotto tabelle separate per percentile

#### Tabella deterministico 100

In [ ]:
# Deterministico --> Dati esistenti (da splittare in subsidenze Pds>0.1-0.001 e inbuti Pds>0.1)

collist = [
       'epoch', 'ril_ksmall', 'ril_klarge', 'tri_ksmall', 'tri_klarge',
       'via_ksmall', 'via_klarge', 'pas_ksmall', 'pas_klarge']

collistred = [
       'ril_ksmall', 'ril_klarge', 'tri_ksmall', 'tri_klarge',
       'via_ksmall', 'via_klarge', 'pas_ksmall', 'pas_klarge']

df_grouped = pd.DataFrame({})

for hh in range (int(1000 / (div_settori * m_punti) * pkfinale)):
    df_temp = df0_100[df0_100.sector2000_50==hh][collist].groupby('epoch').sum()
    
    if df_temp.shape[0]==0: df_temp= pd.DataFrame(np.zeros((1,len(collistred))), columns=collistred)
        
    df_temp = df_temp.replace(np.nan, 0).apply(pd.to_numeric)
    df_temp['index'] = hh
    df_temp['sector2000_50'] = hh
    ###
    df_temp['vol_dave'] = df0_100[df0_100.sector2000_50==hh][['epoch','vol_dave']].groupby('epoch').sum().astype('int32')
    '''
    attenzione nel df_zo stochastico 
              dbase : diametro al piano campagna e 
              dave: diametro al contatto calcari/copertura
    nel df0 invece è il contrario ma mi adatto al df_zo
    '''

    df_temp['n_karst_2000d'] = df0[(df0.sector2000_50==hh)
                                  ][['sector2000_50']].count()[0]
    df_temp['n_karst_100d'] = df0_100[(df0_100.sector2000_50==hh)
                                  ][['sector2000_50']].count()[0]
    

    df_temp['dbase'] = df0_100[df0_100.sector2000_50==hh][['epoch','dave']].groupby('epoch').mean().astype('int32')
    df_temp['dave'] = df0_100[df0_100.sector2000_50==hh][['epoch','dbase']].groupby('epoch').mean().astype('int32')
    
    df_temp.set_index('index', inplace=True)
    df_grouped = df_grouped.append(df_temp)


df_grouped['viaducts'] = df_grouped.via_klarge + df_grouped.via_ksmall
df_grouped['bridges'] = df_grouped.pas_klarge + df_grouped.pas_ksmall
df_grouped = df_grouped[['sector2000_50','dbase','dave','vol_dave','ril_klarge', 'ril_ksmall', 'tri_klarge',
       'tri_ksmall', 'viaducts', 'bridges',"n_karst_2000d","n_karst_100d"]]

df_grouped_determ = df_grouped.fillna(0)
df_grouped_determ.to_csv('df_grouped_determ.csv', sep=';')
del df_grouped
df_grouped_determ.head(4)

#### Karst superficiali e profondi nel df0   --> per stimare riempimento cavità m3

In [ ]:
df_vol_sup = df0[df0.check_100==True][['sector2000_50','volume']].groupby('sector2000_50').sum()
df_vol_sup.reset_index(inplace=True)
df_vol_sup.rename(columns={df_vol_sup.columns.tolist()[-1]:'df0_volk_superf_100'}, inplace=True)
df_vol_sup.to_csv('df_vol_sup.csv', sep=";")
display(df_vol_sup.head(3))
df_vol_prof = df0[df0.check_100==True][['sector2000_50','vol_dbase']].groupby('sector2000_50').sum()
df_vol_prof.reset_index(inplace=True)
df_vol_prof.rename(columns={df_vol_prof.columns.tolist()[-1]:'df0_volk_prof_100'}, inplace=True)
df_vol_prof.to_csv('df_vol_prof.csv', sep=";")
display(df_vol_prof.head(3))

#### Creo la funzione per creare le tabelle

In [ ]:
# from df_grouped95 -> to -> df_grouped_95_t ovvero completato con altre informazioni
# from df_g50 ['sector2000_50', 'Litho', 'Tipo_plus','h_rilevato', 'h_rilevato_true', 'h_trincea', 'h_viadotto', 'h_copertura',
# 'h_copertura_ril', 'h_copertura_tri', 'h_copertura_via', 'LRilevato', 'LTrincea', 'LViaduct', 'LBridge', 'n_karst', 'sector_area']
# n_karst_2000_t sono i # karst nella banda 2km sia da deterministico (analisi karst in superficie) che da stochastico q95
# n_karst_100_t sono i # karst come sopra ma su banda 100m
# ril-klarge ril-ksmall tri_klarge tri_ksmall viaducts bridges sono i numeri di karst nella banda 100m


def createdfgrouped(df_zo_100, df_zo, df_grouped_determ, df_g50, df_vol_prof, df_vol_sup, _quantile):
    
    collist = [
           'epoch', 'ril_ksmall', 'ril_klarge', 'tri_ksmall', 'tri_klarge',
           'via_ksmall', 'via_klarge', 'pas_ksmall', 'pas_klarge']
    
    list_col_g50 = ['sector2000_50','Litho','Tipo_plus','h_rilevato', 'h_rilevato_true','h_trincea','h_viadotto','h_copertura','h_copertura_ril','h_copertura_tri','h_copertura_via',
                'LRilevato', 'LTrincea', 'LViaduct', 'LBridge','n_karst', 'sector_area']

    df_grouped = pd.DataFrame({})

    for hh in range (int(1000 / (div_settori * m_punti) * pkfinale)):
        df_temp = df_zo_100[(df_zo_100.sector2000_50==hh)][collist].groupby('epoch').sum().quantile(_quantile).round(0)
        df_temp['index'] = hh
        df_temp['sector2000_50'] = hh

        df_temp['vol_dave'] = df_zo_100[(df_zo_100['check_100']==True) & (df_zo_100.sector2000_50==hh)
                                       ][['epoch','vol_dave']].groupby('epoch').sum().mean()[0].round(2)
        
        df_temp['h_copertura'] = df_zo_100[df_zo_100.sector2000_50==hh][['epoch','h_copertura']
                                                                       ].groupby('epoch').mean().mean()[0].round(2)
        
        df_temp['dbase'] = df_zo_100[df_zo_100.sector2000_50==hh][['epoch','dbase']].groupby('epoch').mean().mean()[0].round(2)
        df_temp['dave'] = df_zo_100[df_zo_100.sector2000_50==hh][['epoch','dave']].groupby('epoch').mean().mean()[0].round(2)
        # df_temp = pd.to_numeric(df_temp.replace(np.nan, 0), downcast='integer')
        
        
        # tolta condizione df_zo.Pds >=0.01 per inserirla deve essere congruente anche nelle righe sopra
        # ad esempio in karst_via kars_ril ect
        df_temp['n_karst_2000'] = df_zo[(df_zo.sector2000_50==hh)&(df_zo.dbase>0)
                                          ][['sector2000_50','epoch']].groupby('epoch').count().quantile(_quantile)[0]
        df_temp['n_karst_100'] = df_zo_100[(df_zo.sector2000_50==hh)&(df_zo_100.dbase>0)
                                  ][['sector2000_50','epoch']].groupby('epoch').count().quantile(_quantile)[0]
        df_grouped = df_grouped.append(df_temp)

    df_grouped.set_index('index', inplace=True)
    df_grouped['viaducts'] = df_grouped.via_klarge + df_grouped.via_ksmall
    df_grouped['bridges'] = df_grouped.pas_klarge + df_grouped.pas_ksmall
    df_grouped = df_grouped[['sector2000_50','dave','dbase','vol_dave','h_copertura','ril_klarge', 'ril_ksmall', 'tri_klarge',
           'tri_ksmall', 'viaducts', 'bridges','n_karst_2000','n_karst_100']]
    

    ###
    
    cols_totsum = ['vol_dave','ril_klarge', 'ril_ksmall', 'tri_klarge', 'tri_ksmall', 'viaducts', 'bridges']

    # somma algebrica dei due dataframe
    df_temp = df_grouped[cols_totsum] + df_grouped_determ[cols_totsum]
    # correggo le colonne che non devono essere sommate
    df_temp[['dbase','dave','sector2000_50','n_karst_2000','n_karst_100']] = df_grouped[[
        'dbase','dave','sector2000_50','n_karst_2000','n_karst_100']]
    df_temp[['n_karst_2000d','n_karst_100d']] = df_grouped_determ[['n_karst_2000d','n_karst_100d']]

    df_temp = pd.merge(df_temp, df_g50[list_col_g50], on='sector2000_50', left_index=True)
    
    # da eliminare
    # n_karst arriva da df_g50
    # df_temp = pd.merge(df_temp, df_grouped_2000[['sector2000_50','n_karst_2000_95']], on='sector2000_50', left_index=True)
    # df_temp['n_karst_2000_95_t'] = df_temp['n_karst'] + df_temp['n_karst_2000_95']
    
    # sostituire t sta per totale
    df_temp['n_karst_2000_t'] = df_temp['n_karst_2000d'] + df_temp['n_karst_2000']
    df_temp['n_karst_100_t'] = df_temp['n_karst_100d'] + df_temp['n_karst_100']
    
    #
    #df_temp = df_temp.drop(columns=['n_karst','n_karst_2000_95'])
    df_temp['nk_kmq_100y'] = df_temp.n_karst_2000_t / df_temp.sector_area / vitautile
    df_temp['nk_kmq_100y'] = df_temp['nk_kmq_100y'].round(3)
    
    # no perché al centro dell'offset hai molto meno errore
    #df_temp['nk_kmq_100y_100'] = df_temp.n_karst_100_t / (df_temp.sector_area/2.5*0.1) / vitautile
    df_temp['nk_kmq_100y_100'] = df_temp.n_karst_100_t / 0.2 / vitautile
    df_temp['nk_kmq_100y_100'] = df_temp['nk_kmq_100y_100'].round(3)
    

    df_temp['LViabri'] = df_temp.LBridge + df_temp.LViaduct
    df_temp['nk_viabri'] = df_temp.viaducts + df_temp.bridges
    df_temp['nk_riltri'] = df_temp[['ril_klarge', 'ril_ksmall', 'tri_klarge', 'tri_ksmall']].sum(axis = 1)
    
    # nk_tot_100 è più alto a causa di approssimazioni per eccesso nel numero di carst per tipologia
    df_temp['nk_tot_100'] = df_temp[['nk_viabri','nk_riltri']].sum(axis = 1)
    df_temp['vol_dave_check'] = 4/3*np.pi*(df_temp.dave/2)**3 / 2 * df_temp['nk_tot_100']
    df_temp = df_temp.merge(df_vol_sup, on='sector2000_50', how='left')
    df_temp = df_temp.merge(df_vol_prof, on='sector2000_50', how='left')
    
    
    # arrotondamenti
    df_temp['h_rilevato'] = df_temp['h_rilevato'].round(2)
    df_temp['sector_area'] = df_temp['sector_area'].round(2)
    df_temp['h_copertura'] = df_temp['h_copertura'].round(2)
    df_temp['h_copertura_ril'] = df_temp['h_copertura_ril'].round(2)
    df_temp['h_copertura_tri'] = df_temp['h_copertura_tri'].round(2)
    df_temp['h_copertura_via'] = df_temp['h_copertura_via'].round(2)
    df_temp['h_trincea'] = df_temp['h_trincea'].round(2)
    df_temp['h_viadotto'] = df_temp['h_viadotto'].round(2)
    
    return df_temp

#### Creo le tabelle applico la funzione

In [ ]:
### applica funzione
df_grouped_95_t = createdfgrouped(df_zo_100, df_zo, df_grouped_determ, df_g50, df_vol_prof, df_vol_sup, 0.95)


### Viadotti ###

df_via_grouped_95_t = df_grouped_95_t[['Tipo_plus','sector2000_50','LViabri','nk_viabri',
                                       'h_viadotto','h_copertura_via','nk_kmq_100y']]
df_via_grouped_95_t = df_via_grouped_95_t.groupby('Tipo_plus').agg({
    'sector2000_50':'first','LViabri':'sum','nk_viabri':'sum','h_viadotto':'max','h_copertura_via':'mean','nk_kmq_100y':'mean'
    })

df_via_grouped_95_t = df_via_grouped_95_t[(df_via_grouped_95_t.index != 'Rilevato') & (df_via_grouped_95_t.index != 'Trincea')]
df_via_grouped_95_t['Tipo_viabri'] = np.where(df_via_grouped_95_t.h_viadotto >= lim_via_bridge, 'viaduct', 'bridge')

### ultimi aggiustamenti ####

df_grouped_95_t = df_grouped_95_t.fillna(0)
df_via_grouped_95_t = df_via_grouped_95_t.fillna(0)

df_grouped_95_t.head(4)
display(df_grouped_95_t.head(4))
display(df_via_grouped_95_t.head(4))

##### Quantile 75   --> df_grouped_75_t & df_via_grouped_75_t

In [ ]:
### applica funzione
df_grouped_75_t = createdfgrouped(df_zo_100, df_zo, df_grouped_determ, df_g50, df_vol_prof, df_vol_sup, 0.75)


### Viadotti ###

df_via_grouped_75_t = df_grouped_75_t[['Tipo_plus','sector2000_50','LViabri','nk_viabri',
                                       'h_viadotto','h_copertura_via','nk_kmq_100y']]
df_via_grouped_75_t = df_via_grouped_75_t.groupby('Tipo_plus').agg({
    'sector2000_50':'first','LViabri':'sum','nk_viabri':'sum','h_viadotto':'max','h_copertura_via':'mean','nk_kmq_100y':'mean'
    })

df_via_grouped_75_t = df_via_grouped_75_t[(df_via_grouped_75_t.index != 'Rilevato') & (df_via_grouped_75_t.index != 'Trincea')]
df_via_grouped_75_t['Tipo_viabri'] = np.where(df_via_grouped_75_t.h_viadotto >= lim_via_bridge, 'viaduct', 'bridge')

### ultimi aggiustamenti ####

df_grouped_75_t = df_grouped_75_t.fillna(0)
df_via_grouped_75_t = df_via_grouped_75_t.fillna(0)

df_grouped_75_t.head(4)
display(df_grouped_75_t.head(4))
display(df_via_grouped_75_t.head(4))

##### Quantile 50   --> df_grouped_50_t & df_via_grouped_50_t

In [ ]:
### applica funzione
df_grouped_50_t = createdfgrouped(df_zo_100, df_zo, df_grouped_determ, df_g50, df_vol_prof, df_vol_sup, 0.50)


### Viadotti ###

df_via_grouped_50_t = df_grouped_50_t[['Tipo_plus','sector2000_50','LViabri','nk_viabri',
                                       'h_viadotto','h_copertura_via','nk_kmq_100y']]
df_via_grouped_50_t = df_via_grouped_50_t.groupby('Tipo_plus').agg({
    'sector2000_50':'first','LViabri':'sum','nk_viabri':'sum','h_viadotto':'max','h_copertura_via':'mean','nk_kmq_100y':'mean'
    })

df_via_grouped_50_t = df_via_grouped_50_t[(df_via_grouped_50_t.index != 'Rilevato') & (df_via_grouped_50_t.index != 'Trincea')]
df_via_grouped_50_t['Tipo_viabri'] = np.where(df_via_grouped_50_t.h_viadotto >= lim_via_bridge, 'viaduct', 'bridge')

### ultimi aggiustamenti ####

df_grouped_50_t = df_grouped_50_t.fillna(0)
df_via_grouped_50_t = df_via_grouped_50_t.fillna(0)

df_grouped_50_t.head(4)
display(df_grouped_50_t.head(4))
display(df_via_grouped_50_t.head(4))

#### Quantile 25 --> df_grouped_25_t & df_via_grouped_25_t

In [ ]:
### applica funzione
df_grouped_25_t = createdfgrouped(df_zo_100, df_zo, df_grouped_determ, df_g50, df_vol_prof, df_vol_sup, 0.25)


### Viadotti ###

df_via_grouped_25_t = df_grouped_25_t[['Tipo_plus','sector2000_50','LViabri','nk_viabri',
                                       'h_viadotto','h_copertura_via','nk_kmq_100y']]
df_via_grouped_25_t = df_via_grouped_25_t.groupby('Tipo_plus').agg({
    'sector2000_50':'first','LViabri':'sum','nk_viabri':'sum','h_viadotto':'max','h_copertura_via':'mean','nk_kmq_100y':'mean'
    })

df_via_grouped_25_t = df_via_grouped_25_t[(df_via_grouped_25_t.index != 'Rilevato') & (df_via_grouped_25_t.index != 'Trincea')]
df_via_grouped_25_t['Tipo_viabri'] = np.where(df_via_grouped_25_t.h_viadotto >= lim_via_bridge, 'viaduct', 'bridge')

### ultimi aggiustamenti ####

df_grouped_25_t = df_grouped_25_t.fillna(0)
df_via_grouped_25_t = df_via_grouped_25_t.fillna(0)

df_grouped_25_t.head(4)
display(df_grouped_25_t.head(4))
display(df_via_grouped_25_t.head(4))

## CALCOLO GLI INTERVENTI

#### creo le funzioni per calcolare gli interventi

In [ ]:

####       sostituisci h_rilevato con h_copertura_ril h_copertura_tri

# trattamento rilevato_tipo per numero di karst per tratta
# ridotto il 17/3 prima era 200
tratt_pernk = 100

def ril_tipo12b(row):
    
    if ((row['h_copertura_ril']<10) & (row['nk_kmq_100y']>=0.01)):
        # rilevati
        ltot1 = (int(row['ril_klarge']) + int(row['ril_ksmall'])) * tratt_pernk
        if ltot1 > row['LRilevato'] : ltot1 = row['LRilevato']
    else:
        ltot1 = 0
    
    if ((row['h_copertura_tri']<10) & (row['nk_kmq_100y']>=0.01)):
        # trincee
        ltot2 = (int(row['tri_klarge']) + int(row['tri_ksmall'])) * tratt_pernk
        if ltot2 > row['LTrincea'] : ltot2 = row['LTrincea']
    else:
        ltot2=0
    
    ltot = ltot1+ltot2
    
    return ltot

def ril_tipo2(row):
    # rilevati
    if ((row['h_copertura_ril']<10)&(row['nk_kmq_100y']>=0.01)&(row['nk_kmq_100y']<0.1)) or \
    ((row.h_copertura_ril >= 10)&(row.h_copertura_ril < 15)&(row['nk_kmq_100y']>=0.01)) :
        # rilevati
        ltot1 = int(row['ril_klarge']) * tratt_pernk + int(row['ril_ksmall']) * tratt_pernk
        if ltot1 > row['LRilevato'] : ltot1 = row['LRilevato']
    else:
        ltot1=0
    # trincee
    if ((row['h_copertura_tri']<10)&(row['nk_kmq_100y']>=0.01)&(row['nk_kmq_100y']<0.1)) or \
    ((row.h_copertura_tri >= 10)&(row.h_copertura_tri < 15)&(row['nk_kmq_100y']>=0.01)) :
        # trincee
        ltot2 = int(row['tri_klarge']) * tratt_pernk + int(row['tri_ksmall']) * tratt_pernk
        if ltot2 > row['LTrincea'] : ltot2 = row['LTrincea']
    else:
        ltot2=0
    
    ltot = ltot1+ltot2
    return ltot

def ril_tipo3(row):
    
    if ((row['h_copertura_ril'] >= 15) & (row['h_copertura_ril'] < 30) & (row['nk_kmq_100y']>=0.01)):
        # rilevati
        ltot1 = int(row['ril_klarge']) * tratt_pernk + int(row['ril_ksmall']) * tratt_pernk
        if ltot1 > row['LRilevato'] : ltot1 = row['LRilevato']
    else:
        ltot1 = 0
        
    if ((row['h_copertura_tri'] >= 15) & (row['h_copertura_tri'] < 30) & (row['nk_kmq_100y']>=0.01)):
        # trincee
        ltot2 = int(row['tri_klarge']) * tratt_pernk + int(row['tri_ksmall']) * tratt_pernk
        if ltot2 > row['LTrincea'] : ltot2 = row['LTrincea'] 
    else:
        ltot2=0
        
    ltot = ltot1+ltot2
    return ltot

def ril_tipo4(row):
    
    if ((row['h_copertura_ril'] >= 30) & (row['nk_kmq_100y']>=0.01)):
        # rilevati
        ltot1 = int(row['ril_klarge']) * tratt_pernk + int(row['ril_ksmall']) *tratt_pernk
        if ltot1 > row['LRilevato'] : ltot1 = row['LRilevato']
    else:
        ltot1=0
    
    if ((row['h_copertura_tri'] >= 30) & (row['nk_kmq_100y']>=0.01)):
        # trincee
        ltot2 = int(row['tri_klarge']) * tratt_pernk + int(row['tri_ksmall']) * tratt_pernk
        if ltot2 > row['LTrincea'] : ltot2 = row['LTrincea']
    else:
        ltot2=0
    
    ltot = ltot1+ltot2
    return ltot


def via_tipo1(row):
    
    if row['h_copertura_via']<6:
        ltot = 1       
    elif (row['Tipo_viabri']=='viaduct')&(row['nk_kmq_100y']<0.01)&(row['h_copertura_via']<10)&(row['h_copertura_via']>=6) :
        ltot=1
    else:
        ltot = 0
        
    return ltot

def via_tipo24(row):
    
    if (row['Tipo_viabri']=='bridge') & (row['h_copertura_via'] >= 6) & (row['h_copertura_via'] < 10) & (row['nk_kmq_100y']>=0.01):
        ltot = 1           
    else:
        ltot = 0
        
    return ltot

def via_tipo34(row):
    
    if (row['Tipo_viabri']=='viaduct') & (row['h_copertura_via'] >= 6) & (row['h_copertura_via'] <20) & (row['nk_kmq_100y']>=0.01):
        ltot = 1           
    else:
        ltot = 0
        
    return ltot
        
def via_tipo5(row):
    
    if (row['Tipo_viabri']=='viaduct') & (row['h_copertura_via'] >= 10) & (row['h_copertura_via'] < 20) & (row['nk_kmq_100y'] < 0.01):
        ltot = 1
   
    elif (row['Tipo_viabri']=='viaduct') & (row['h_copertura_via'] >= 20):
        ltot = 1
        
    elif (row['Tipo_viabri']=='bridge') & (row['h_copertura_via'] >= 6) & (row['h_copertura_via'] < 10) & (row['nk_kmq_100y'] < 0.01):
        ltot = 1
        
    elif (row['Tipo_viabri']=='bridge') & (row['h_copertura_via'] >= 10):
        ltot = 1
        
    else:
        ltot = 0        
    return ltot

# def ril_geogriglia(row):
    
#     # aggiungere in assenza di interventi di consolidamento*****************
    
#     if (row['h_copertura'] >= 15) & (row['h_copertura'] < 30) & (row['nk_kmq_100y'] > 0.01):pass
    
def ril_compactg(row):
    # da aggiornare con proporzione tra subsidenze e inghiottitoi
    #if (row['h_copertura']<30) & (row['h_copertura']>=10) & (row['nk_kmq_100y'] > 0.1) & ((row['ril_klarge']+row['tri_klarge'])>0):
    #il vincolo sulle klarge non è da considerare
    if row['nk_kmq_100y'] > 0.1:
        try:
            prop = (row['ril_klarge']+row['tri_klarge'])/((row['ril_klarge']+row['tri_klarge']+row['ril_ksmall']+row['tri_ksmall']))
            #ltot = (row['ril_tipo2'] + row['ril_tipo3']) * prop
            #17/03 impostato solo sul rilevato tipo2
            ltot = (row['ril_tipo2']) * prop
        except:
            ltot=0
    else:
        ltot = 0
        
    return round(ltot, 1)

def ril_gratic(row):
    
    try:
        # proporzione di subsidenze con inghiottitoi (da aggiornare con quantità in funziuone del Pds)
        prop = (row['ril_klarge']+row['tri_klarge'])/((row['ril_klarge']+row['tri_klarge']+row['ril_ksmall']+row['tri_ksmall']))
        ltot = row['ril_tipo12b'] * (1-prop)
    except:
        ltot = 0

    return round(ltot, 1)


def ril_vibrof(row):
    
    if row['Litho'] == 's':
        try:
            # proporzione di subsidenze con inghiottitoi (da aggiornare con quantità in funziuone del Pds)
            prop = (row['ril_klarge']+row['tri_klarge'])/((row['ril_klarge']+row['tri_klarge']+row['ril_ksmall']+row['tri_ksmall']))
            ltot = row['ril_tipo12b'] * prop
        except:
            ltot = 0
    else:
        ltot = 0
    
    return round(ltot, 1)


def ril_dyncomp(row):
    
#     if ((row['h_copertura']<10)&(row['nk_kmq_100y']>=0.01)&(row['nk_kmq_100y']<0.1) & (row['Litho']=='s')):
#         ltot = row['ril_tipo2']

    if row['Litho'] == 'a':
        try:
            # proporzione di subsidenze con inghiottitoi (da aggiornare con quantità in funziuone del Pds)
            prop = (row['ril_klarge']+row['tri_klarge'])/((row['ril_klarge']+row['tri_klarge']+row['ril_ksmall']+row['tri_ksmall']))
            ltot = row['ril_tipo12b'] * prop
        except:
            ltot = 0
    else:
        ltot = 0
    
    return round(ltot, 1)


def geogri(row):
    
    # applico nei rilevati e trincee tipo 4 dove la copertura è inferiore a 50m
    # aggiunto limite di applicazione lambda > 0.05 e riduzione di 50% ovunque 16/03/2021
    # rilevati
    if ((row.ril_tipo4) > 0) & (row.h_copertura_ril < 50) & (row.nk_kmq_100y > 0.05):
        ltot1 = row.ril_tipo4 * row.LRilevato/(row.LRilevato + row.LTrincea) * 0.5
    else:
        ltot1 = 0
    
    # trincee
    if ((row.ril_tipo4) > 0) & (row.h_copertura_tri < 50) & (row.nk_kmq_100y > 0.05):
        ltot2 = row.ril_tipo4 * row.LTrincea/(row.LRilevato + row.LTrincea) * 0.5
    else:
        ltot2 = 0
    
    # applico nelle zone ril tipo 1-2-3 dove non ci sono interneventi
    # no compctgr no vibrof no dyncomp no graticcio
    ltota = row['ril_tipo12b'] + row['ril_tipo2'] + row['ril_tipo3']
    ltotb = row['ril_compactg'] + row['ril_vibrof'] + row['ril_dyncomp'] + row['ril_gratic']
    ltot3 = ltota - ltotb
    if ltot3 < 0 : ltot3 = 0
    
    ltot = round(ltot1+ltot2+ltot3,1)
    
    return ltot

def kgrout_vol_sup(row):
    
    if row.ril_gratic > 0:
        vtot = row.df0_volk_superf_100
    else:
        vtot = 0
        
    return round(vtot,1)

def kgrout_vol(row):
    # in quanto nello stesso km posso avere più di un intervento (non sovrapposto ma consecutivo)
    # devo calcolare la percentuale di area che ricade negli interventi che prevedono riempimenti
    # aggiunta sconto 0.5 su tutte, ulteriore 0.5 se lambda<0.1 e 0 se lambda < 0.05 16/03/2021
    try:
        prop1 = (row.ril_gratic + row.geogri)/(row.ril_gratic + row.ril_compactg + row.ril_vibrof + row.ril_dyncomp + row.geogri)
    except:
        prop1 = 0
        
    prop2 = (30 - row.h_copertura)/30
    if prop2 < 0 : prop2 = 0
    
    # 17/03
    # cavità identificate iniziali da evidenze in superficie
    v1 = row.df0_volk_prof_100
    # cavità stimate senza evidenze in superficie (stimata sulla fascia 100m)
    v2 = row.vol_dave - row.df0_volk_prof_100  
    vtot = (v1 + v2 * 0.1) * prop1 * prop2 * 0.5
    if row.nk_kmq_100y < 0.1 : vtot *= 0.5
    if row.nk_kmq_100y < 0.05 : vtot = 0

       
    # condizione 16/03
    #vtot = row.vol_dave * prop1 * prop2 * 0.5
    #if row.nk_kmq_100y < 0.1 : vtot *= 0.5
    #if row.nk_kmq_100y < 0.05 : vtot = 0.0
    
    return round(vtot,1)

def l_dreni(row):
    
    passo_dreni = 20
    lati = 2
    
    try:
        h_copertura_media = (row.h_copertura_ril * row.LRilevato + row.h_copertura_tri * row.LTrincea)/(row.LRilevato + row.LTrincea)
    except:
        h_copertura_media = 0
    
    # lunghezza dreno sommata
    #ltot = ((row.ril_tipo12b - row.ril_gratic) + row.ril_tipo2 + row.ril_tipo3)/passo_dreni*lati*h_copertura_media
    # lunghezza di applicazione lungo il tracciato
    #ltot = ((row.ril_tipo12b - row.ril_gratic) + row.ril_tipo2 + row.ril_tipo3)
    # 17/03 riunione interna
    #ltot = (row.ril_compactg + row.ril_vibrof + row.ril_dyncomp)
    #17/10 riunione --> ril_vibrof e ril_dyncomp non si fanno al loro posto RC slab
    ltot = (row.ril_compactg)
    
    
    #  condizione aggiunta meeting 16/03/2021
    # non piu valida
    #if row.nk_kmq_100y < 0.1 : ltot *= 0.5
    #if row.nk_kmq_100y < 0.05 : ltot = 0
    
    return ltot


def l_trincea(row):
    
    # trincee sempre entrambe i lati quanto copertura < 20m
    # nota se metto row.ril_tipo12b vuol dire che automaticamente escludo coperture >10m!
    if (row.h_copertura_tri < 20) : 
        ltot1 = row.ril_tipo12b * row.LTrincea/(row.LRilevato + row.LTrincea) * 2
    else:
        ltot1 = 0
        
    # rilevati sempre un lato in base alla pendenza quando h<10
    # un lato e quando h 10-20m per le zone ad altissimo rischio lambda > 0.3
    # nota se metto row.ril_tipo12b vuol dire che automaticamente escludo coperture >10m!
    if (row.h_copertura_ril < 10): 
        ltot2 = row.ril_tipo12b * row.LRilevato/(row.LRilevato + row.LTrincea) * 1
    elif ((row.h_copertura_ril >= 10) & (row.h_copertura_ril < 20) & (row.nk_kmq_100y > 0.3)):
        ltot2 = row.ril_tipo12b * row.LRilevato/(row.LRilevato + row.LTrincea) * 1
    else:
        ltot2 = 0  
        
    ltot = ltot1 + ltot2
    
    return ltot        

#### Applico le funzioni PER 95

In [ ]:
df_grouped_95_t['ril_tipo12b'] = df_grouped_95_t.apply(ril_tipo12b, axis=1)
df_grouped_95_t['ril_tipo2'] = df_grouped_95_t.apply(ril_tipo2, axis=1)
df_grouped_95_t['ril_tipo3'] = df_grouped_95_t.apply(ril_tipo3, axis=1)
df_grouped_95_t['ril_tipo4'] = df_grouped_95_t.apply(ril_tipo4, axis=1)

df_grouped_95_t = df_grouped_95_t.fillna(0)


df_via_grouped_95_t['via_tipo1'] = df_via_grouped_95_t.apply(via_tipo1, axis=1)
df_via_grouped_95_t['via_tipo24'] = df_via_grouped_95_t.apply(via_tipo24, axis=1)
df_via_grouped_95_t['via_tipo34'] = df_via_grouped_95_t.apply(via_tipo34, axis=1)
df_via_grouped_95_t['via_tipo5'] = df_via_grouped_95_t.apply(via_tipo5, axis=1)

df_via_grouped_95_t = df_via_grouped_95_t.fillna(0)

df_grouped_95_t['ril_compactg'] = df_grouped_95_t.apply(ril_compactg, axis=1)
df_grouped_95_t['ril_vibrof'] = df_grouped_95_t.apply(ril_vibrof, axis=1)
df_grouped_95_t['ril_dyncomp'] = df_grouped_95_t.apply(ril_dyncomp, axis=1)
df_grouped_95_t['ril_gratic'] = df_grouped_95_t.apply(ril_gratic, axis=1)
df_grouped_95_t['ril_gratic_vibr_dyn'] = df_grouped_95_t[['ril_gratic','ril_vibrof','ril_dyncomp']].sum(axis=1)
df_grouped_95_t['geogri'] = df_grouped_95_t.apply(geogri, axis=1)
df_grouped_95_t['kgrout_vol'] = df_grouped_95_t.apply(kgrout_vol, axis=1)
df_grouped_95_t['kgrout_vol_sup'] = df_grouped_95_t.apply(kgrout_vol_sup, axis=1)
df_grouped_95_t['l_dreni'] = round(df_grouped_95_t.apply(l_dreni, axis=1),0)
df_grouped_95_t['l_trincea'] = round(df_grouped_95_t.apply(l_trincea, axis=1),0)

df_grouped_95_t = df_grouped_95_t.fillna(0)

display(df_grouped_95_t.head(4))

df_temp = df_via_grouped_95_t[['sector2000_50','Tipo_viabri','h_copertura_via',
       'h_viadotto', 'LViabri', 'nk_kmq_100y', 'via_tipo1', 'via_tipo24',
       'via_tipo34', 'via_tipo5']]
df_temp.to_csv('df_via_grouped_95_t.csv',sep=';')
display(df_temp.head(10))

df_temp = df_grouped_95_t[(df_grouped_95_t.ril_tipo12b>0) | (df_grouped_95_t.ril_tipo2>0) | (df_grouped_95_t.ril_tipo3>0) | (df_grouped_95_t.ril_tipo4>0)][['sector2000_50',
       'h_copertura','h_copertura_ril', 'h_copertura_tri','Litho',
       'LRilevato', 'LTrincea',
       'nk_kmq_100y', 'ril_tipo12b',
       'ril_tipo2', 'ril_tipo3', 'ril_tipo4', 'ril_gratic','ril_gratic_vibr_dyn','ril_compactg', 'ril_vibrof', 'ril_dyncomp','geogri',
       'kgrout_vol_sup','kgrout_vol','l_dreni','l_trincea']].fillna(0)
df_temp.to_csv('df_grouped_95_t.csv', sep=';')
display(df_temp.head(41))

#### CALCOLO INTERVENTI q75

In [ ]:
df_grouped_75_t['ril_tipo12b'] = df_grouped_75_t.apply(ril_tipo12b, axis=1)
df_grouped_75_t['ril_tipo2'] = df_grouped_75_t.apply(ril_tipo2, axis=1)
df_grouped_75_t['ril_tipo3'] = df_grouped_75_t.apply(ril_tipo3, axis=1)
df_grouped_75_t['ril_tipo4'] = df_grouped_75_t.apply(ril_tipo4, axis=1)

df_grouped_75_t = df_grouped_75_t.fillna(0)


df_via_grouped_75_t['via_tipo1'] = df_via_grouped_75_t.apply(via_tipo1, axis=1)
df_via_grouped_75_t['via_tipo24'] = df_via_grouped_75_t.apply(via_tipo24, axis=1)
df_via_grouped_75_t['via_tipo34'] = df_via_grouped_75_t.apply(via_tipo34, axis=1)
df_via_grouped_75_t['via_tipo5'] = df_via_grouped_75_t.apply(via_tipo5, axis=1)

df_via_grouped_75_t = df_via_grouped_75_t.fillna(0)

df_grouped_75_t['ril_compactg'] = df_grouped_75_t.apply(ril_compactg, axis=1)
df_grouped_75_t['ril_vibrof'] = df_grouped_75_t.apply(ril_vibrof, axis=1)
df_grouped_75_t['ril_dyncomp'] = df_grouped_75_t.apply(ril_dyncomp, axis=1)
df_grouped_75_t['ril_gratic'] = df_grouped_75_t.apply(ril_gratic, axis=1)
df_grouped_75_t['ril_gratic_vibr_dyn'] = df_grouped_75_t[['ril_gratic','ril_vibrof','ril_dyncomp']].sum(axis=1)
df_grouped_75_t['geogri'] = df_grouped_75_t.apply(geogri, axis=1)
df_grouped_75_t['kgrout_vol'] = df_grouped_75_t.apply(kgrout_vol, axis=1)
df_grouped_75_t['kgrout_vol_sup'] = df_grouped_75_t.apply(kgrout_vol_sup, axis=1)
df_grouped_75_t['l_dreni'] = round(df_grouped_75_t.apply(l_dreni, axis=1),0)
df_grouped_75_t['l_trincea'] = round(df_grouped_75_t.apply(l_trincea, axis=1),0)

df_grouped_75_t = df_grouped_75_t.fillna(0)

display(df_grouped_75_t.head(4))

df_temp = df_via_grouped_75_t[['sector2000_50','Tipo_viabri','h_copertura_via',
       'h_viadotto', 'LViabri', 'nk_kmq_100y', 'via_tipo1', 'via_tipo24',
       'via_tipo34', 'via_tipo5']]
df_temp.to_csv('df_via_grouped_75_t.csv',sep=';')
display(df_temp.head(4))

df_temp = df_grouped_75_t[(df_grouped_75_t.ril_tipo12b>0) | (df_grouped_75_t.ril_tipo2>0) | (df_grouped_75_t.ril_tipo3>0) | (df_grouped_75_t.ril_tipo4>0)][['sector2000_50',
       'h_copertura','h_copertura_ril', 'h_copertura_tri','Litho',
       'LRilevato', 'LTrincea',
       'nk_kmq_100y', 'ril_tipo12b',
       'ril_tipo2', 'ril_tipo3', 'ril_tipo4', 'ril_gratic','ril_gratic_vibr_dyn','ril_compactg', 'ril_vibrof', 'ril_dyncomp','geogri',
       'kgrout_vol_sup','kgrout_vol','l_dreni','l_trincea']].fillna(0)
df_temp.to_csv('df_grouped_75_t.csv', sep=';')
display(df_temp.head(41))

#### CALCOLO INTERVENTI q50

In [ ]:
df_grouped_50_t['ril_tipo12b'] = df_grouped_50_t.apply(ril_tipo12b, axis=1)
df_grouped_50_t['ril_tipo2'] = df_grouped_50_t.apply(ril_tipo2, axis=1)
df_grouped_50_t['ril_tipo3'] = df_grouped_50_t.apply(ril_tipo3, axis=1)
df_grouped_50_t['ril_tipo4'] = df_grouped_50_t.apply(ril_tipo4, axis=1)

df_grouped_50_t = df_grouped_50_t.fillna(0)


df_via_grouped_50_t['via_tipo1'] = df_via_grouped_50_t.apply(via_tipo1, axis=1)
df_via_grouped_50_t['via_tipo24'] = df_via_grouped_50_t.apply(via_tipo24, axis=1)
df_via_grouped_50_t['via_tipo34'] = df_via_grouped_50_t.apply(via_tipo34, axis=1)
df_via_grouped_50_t['via_tipo5'] = df_via_grouped_50_t.apply(via_tipo5, axis=1)

df_via_grouped_50_t = df_via_grouped_50_t.fillna(0)

df_grouped_50_t['ril_compactg'] = df_grouped_50_t.apply(ril_compactg, axis=1)
df_grouped_50_t['ril_vibrof'] = df_grouped_50_t.apply(ril_vibrof, axis=1)
df_grouped_50_t['ril_dyncomp'] = df_grouped_50_t.apply(ril_dyncomp, axis=1)
df_grouped_50_t['ril_gratic'] = df_grouped_50_t.apply(ril_gratic, axis=1)
df_grouped_50_t['ril_gratic_vibr_dyn'] = df_grouped_50_t[['ril_gratic','ril_vibrof','ril_dyncomp']].sum(axis=1)
df_grouped_50_t['geogri'] = df_grouped_50_t.apply(geogri, axis=1)
df_grouped_50_t['kgrout_vol'] = df_grouped_50_t.apply(kgrout_vol, axis=1)
df_grouped_50_t['kgrout_vol_sup'] = df_grouped_50_t.apply(kgrout_vol_sup, axis=1)
df_grouped_50_t['l_dreni'] = round(df_grouped_50_t.apply(l_dreni, axis=1),0)
df_grouped_50_t['l_trincea'] = round(df_grouped_50_t.apply(l_trincea, axis=1),0)

df_grouped_50_t = df_grouped_50_t.fillna(0)

display(df_grouped_50_t.head(4))

df_temp = df_via_grouped_50_t[['sector2000_50','Tipo_viabri','h_copertura_via',
       'h_viadotto', 'LViabri', 'nk_kmq_100y', 'via_tipo1', 'via_tipo24',
       'via_tipo34', 'via_tipo5']]
df_temp.to_csv('df_via_grouped_50_t.csv',sep=';')
display(df_temp.head(4))

df_temp = df_grouped_50_t[(df_grouped_50_t.ril_tipo12b>0) | (df_grouped_50_t.ril_tipo2>0) | (df_grouped_50_t.ril_tipo3>0) | (df_grouped_50_t.ril_tipo4>0)][['sector2000_50',
       'h_copertura', 'h_copertura_ril', 'h_copertura_tri','Litho',
       'LRilevato', 'LTrincea',
       'nk_kmq_100y', 'ril_tipo12b',
       'ril_tipo2', 'ril_tipo3', 'ril_tipo4', 'ril_gratic','ril_gratic_vibr_dyn','ril_compactg', 'ril_vibrof', 'ril_dyncomp','geogri',
       'kgrout_vol_sup','kgrout_vol','l_dreni','l_trincea']].fillna(0)
df_temp.to_csv('df_grouped_50_t.csv', sep=';')
display(df_temp.head(41))

#### CALCOLO INTERVENTI q25

In [ ]:
df_grouped_25_t['ril_tipo12b'] = df_grouped_25_t.apply(ril_tipo12b, axis=1)
df_grouped_25_t['ril_tipo2'] = df_grouped_25_t.apply(ril_tipo2, axis=1)
df_grouped_25_t['ril_tipo3'] = df_grouped_25_t.apply(ril_tipo3, axis=1)
df_grouped_25_t['ril_tipo4'] = df_grouped_25_t.apply(ril_tipo4, axis=1)

df_grouped_25_t = df_grouped_25_t.fillna(0)


df_via_grouped_25_t['via_tipo1'] = df_via_grouped_25_t.apply(via_tipo1, axis=1)
df_via_grouped_25_t['via_tipo24'] = df_via_grouped_25_t.apply(via_tipo24, axis=1)
df_via_grouped_25_t['via_tipo34'] = df_via_grouped_25_t.apply(via_tipo34, axis=1)
df_via_grouped_25_t['via_tipo5'] = df_via_grouped_25_t.apply(via_tipo5, axis=1)

df_via_grouped_25_t = df_via_grouped_25_t.fillna(0)

df_grouped_25_t['ril_compactg'] = df_grouped_25_t.apply(ril_compactg, axis=1)
df_grouped_25_t['ril_vibrof'] = df_grouped_25_t.apply(ril_vibrof, axis=1)
df_grouped_25_t['ril_dyncomp'] = df_grouped_25_t.apply(ril_dyncomp, axis=1)
df_grouped_25_t['ril_gratic'] = df_grouped_25_t.apply(ril_gratic, axis=1)
df_grouped_25_t['ril_gratic_vibr_dyn'] = df_grouped_25_t[['ril_gratic','ril_vibrof','ril_dyncomp']].sum(axis=1)
df_grouped_25_t['geogri'] = df_grouped_25_t.apply(geogri, axis=1)
df_grouped_25_t['kgrout_vol'] = df_grouped_25_t.apply(kgrout_vol, axis=1)
df_grouped_25_t['kgrout_vol_sup'] = df_grouped_25_t.apply(kgrout_vol_sup, axis=1)
df_grouped_25_t['l_dreni'] = round(df_grouped_25_t.apply(l_dreni, axis=1),0)
df_grouped_25_t['l_trincea'] = round(df_grouped_25_t.apply(l_trincea, axis=1),0)

df_grouped_25_t = df_grouped_25_t.fillna(0)

display(df_grouped_25_t.head(4))

df_temp = df_via_grouped_25_t[['sector2000_50','Tipo_viabri','h_copertura_via',
       'h_viadotto', 'LViabri', 'nk_kmq_100y', 'via_tipo1', 'via_tipo24',
       'via_tipo34', 'via_tipo5']]
df_temp.to_csv('df_via_grouped_25_t.csv',sep=';')
display(df_temp.head(4))

df_temp = df_grouped_25_t[(df_grouped_25_t.ril_tipo12b>0) | (df_grouped_25_t.ril_tipo2>0) | (df_grouped_25_t.ril_tipo3>0) | (df_grouped_25_t.ril_tipo4>0)][['sector2000_50',
       'h_copertura', 'h_copertura_ril', 'h_copertura_tri','Litho',
       'LRilevato', 'LTrincea',
       'nk_kmq_100y', 'ril_tipo12b',
       'ril_tipo2', 'ril_tipo3', 'ril_tipo4', 'ril_gratic','ril_gratic_vibr_dyn','ril_compactg', 'ril_vibrof', 'ril_dyncomp','geogri',
       'kgrout_vol_sup','kgrout_vol','l_dreni','l_trincea']].fillna(0)
# file per salvataggio con colonne selezionate
df_temp.to_csv('df_grouped_25_t.csv', sep=';')
display(df_temp.head(41))

# ==========================================================

In [ ]:
#### aggiorno df_g50

In [ ]:
df_g50.merge(df_grouped_25_t[['sector2000_50','nk_tot']].rename(columns={'nk_tot':'nk_tot_p25'}), on = 'sector2000_50', how='left')

### PLOTTO DISTRIBUZIONI

In [ ]:
df0.columns

In [ ]:
sns.kdeplot(
    data=df0[df0.type == 'Inbuto'], x="h_copertura", y="dave", hue="Litho", fill=True,
)

In [ ]:
sns.scatterplot(
    data=df0.groupby('sector2000_50').agg({'h_copertura':'mean', 'long':'count', 'type':'max', 'Litho':'max', 'dave':'mean'}), 
    x="long", y="h_copertura", hue='Litho', sizes='dave'
)
plt.xlabel('n° di occorrenze')
plt.xlim(0,160)


In [ ]:
sns.kdeplot(
    data=df0[df0.type == 'Subsidenza'], x="h_copertura", y="dave", hue="Litho", fill=True,
)

In [ ]:
numero_righe = 8
numero_colonne = 5


fig, axs = plt.subplots(numero_righe, numero_colonne, sharex=True, figsize=(18,24))

h=0

for i in range(numero_righe):
    for e in range(numero_colonne):

        histdata1 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().ril_klarge
        histdata2 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().tri_klarge
        histdata = histdata1.append(histdata2)

        sns.kdeplot(histdata, color='grey', fill=True, ax=axs[i, e])
        axs[i, e].set_title('Sector: '+ str(h))

        # Add labels
        axs[i, e].set_xlabel('Number of karst')
        axs[i, e].set_ylabel('Frequency')
        axs[i, e].set_xlim(0,5)
        axs[i, e].set_xticks([0,1,2,3,4,5])
        axs[i, e].set_yticks([])
        _ymin, _ymax = axs[i, e].get_ylim()
        axs[i, e].vlines(np.quantile(histdata,0.25), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.75), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.95), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.50), _ymin, _ymax, colors = 'red', linestyles= 'dashed',  linewidth=2)
        axs[i, e].text(.5,.9,'centered title',
                horizontalalignment='center',
                transform=ax.transAxes)
        h += 1

plt.savefig('kde_klarge.png')
plt.close()


In [ ]:
numero_righe = 8
numero_colonne = 5


fig, axs = plt.subplots(numero_righe, numero_colonne, sharex=True, figsize=(18,24))

h=0

for i in range(numero_righe):
    for e in range(numero_colonne):

        histdata1 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().ril_ksmall
        histdata2 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().tri_ksmall
        histdata = histdata1.append(histdata2)

        sns.kdeplot(histdata, color='grey', fill=True, ax=axs[i, e])
        axs[i, e].set_title('Sector: '+ str(h))

        # Add labels
        axs[i, e].set_xlabel('Number of karst')
        axs[i, e].set_ylabel('Frequency')
        axs[i, e].set_xlim(0,5)
        axs[i, e].set_xticks([0,1,2,3,4,5])
        axs[i, e].set_yticks([])
        _ymin, _ymax = axs[i, e].get_ylim()
        axs[i, e].vlines(np.quantile(histdata,0.25), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.75), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.95), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
        axs[i, e].vlines(np.quantile(histdata,0.50), _ymin, _ymax, colors = 'red', linestyles= 'dashed',  linewidth=2)
        h += 1

plt.savefig('kde_ksmall.png')
plt.close()


#### Viadotti

In [ ]:
numero_righe = 4
numero_colonne = 5


fig, axs = plt.subplots(numero_righe, numero_colonne, sharex=True, figsize=(18,12))

h = 0

for i in range(numero_righe):
    for e in range(numero_colonne):
        try:
            histdata1 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().via_klarge
            histdata2 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().via_ksmall
            histdata3 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().pas_klarge
            histdata4 = df_zo_100[(df_zo_100.sector2000_50 == h) & (df_zo_100['check_100']==True)].groupby('epoch').sum().pas_ksmall
            histdata = histdata1.append(histdata2).append(histdata3).append(histdata4)
            
            sns.histplot(histdata, element='step', stat='density', ax=axs[i, e], color='c')
            sns.kdeplot(histdata, color='grey', fill=True, ax=axs[i, e])
            axs[i, e].set_title('Sector: '+ str(h))

            # Add labels
            axs[i, e].set_xlabel('Number of karst')
            axs[i, e].set_ylabel('Frequency')
            axs[i, e].set_xlim(0,5)
            _ymin, _ymax = axs[i, e].get_ylim()
            axs[i, e].vlines(np.quantile(histdata,0.25), _ymin, _ymax, colors = 'lightgreen', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.75), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.95), _ymin, _ymax, colors = 'red', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.50), _ymin, _ymax, colors = 'k', linestyles= 'dashed',  linewidth=1)
            axs[i, e].text(.5,.9,'centered title',
                horizontalalignment='center',
                transform=axs[i, e].transAxes)
        except:
            pass
        h += 1

plt.savefig('kde_via_pas.png')
plt.close()

In [ ]:
numero_righe = 8
numero_colonne = 5


# fig, axs = plt.subplots(numero_righe, numero_colonne, sharex=True, sharey=True, figsize=(18,12))
fig, axs = plt.subplots(numero_righe, numero_colonne, figsize=(18,30))

h = 0
_xmax = 0.5
_nbins = 5
_ymax = 75

for i in range(numero_righe):
    for e in range(numero_colonne):
        try:
            histdata1 = df_zo[(df_zo.sector2000_50 == h)&(df_zo.dbase>0)].groupby('epoch').count().long
            histdata_det = df_g50[(df_g50.sector2000_50==h)].n_karst
            histdata =  (histdata1 + histdata_det.values[0])/ df_g50.sector_area.mean() /100
            
            sns.histplot(histdata, element='step', stat='density', ax=axs[i, e], color='c')
            sns.kdeplot(histdata, color='grey', fill=True, ax=axs[i, e])
            #axs[i, e].set_title('Sector: '+ str(h))

            # Add labels
            axs[i, e].set_xlabel('Number of karst')
            axs[i, e].set_ylabel('')
            #axs[i, e].set_xlim(0,_xmax)
            #axs[i, e].set_ylim(0,_ymax)
            _ymin, _ymax = axs[i, e].get_ylim()
            axs[i, e].vlines(np.quantile(histdata,0.25), _ymin, _ymax, colors = 'lightgreen', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.75), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.95), _ymin, _ymax, colors = 'red', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.50), _ymin, _ymax, colors = 'k', linestyles= 'dashed',  linewidth=1)
            axs[i, e].text(.5,.9,'Sector: '+ str(h),
                horizontalalignment='center',
                transform=axs[i, e].transAxes)
        except:
            pass
        h += 1
plt.locator_params(axis="x", nbins=_nbins)
plt.savefig('kde_all_2000.png')
plt.close()

In [ ]:
numero_righe = 8
numero_colonne = 5


# fig, axs = plt.subplots(numero_righe, numero_colonne, sharex=True, sharey=True, figsize=(18,12))
fig, axs = plt.subplots(numero_righe, numero_colonne, figsize=(18,30))

h = 0
_xmax = 5
_nbins = 5

for i in range(numero_righe):
    for e in range(numero_colonne):
        try:
            histdata1 = df_zo[(df_zo.sector2000_50 == h)&(df_zo.check_100 == True)&(df_zo.dbase>0)].groupby('epoch').count().long
            histdata_det = df_g50[(df_g50.sector2000_50==h)].n_karst
            histdata =  (histdata1 + histdata_det.values[0])
            
            sns.histplot(histdata, element='step', stat='density', ax=axs[i, e], color='c')
            sns.kdeplot(histdata, color='grey', fill=True, ax=axs[i, e])
            #axs[i, e].set_title('Sector: '+ str(h))

            # Add labels
            axs[i, e].set_xlabel('Number of karst')
            axs[i, e].set_ylabel('')
            #axs[i, e].set_xlim(0,_xmax)
            #axs[i, e].set_ylim(0,_ymax)
            _ymin, _ymax = axs[i, e].get_ylim()
            axs[i, e].vlines(np.quantile(histdata,0.25), _ymin, _ymax, colors = 'lightgreen', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.75), _ymin, _ymax, colors = 'orange', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.95), _ymin, _ymax, colors = 'red', linestyles= 'dashed',  linewidth=1)
            axs[i, e].vlines(np.quantile(histdata,0.50), _ymin, _ymax, colors = 'k', linestyles= 'dashed',  linewidth=1)
            axs[i, e].text(.5,.9,'Sector: '+ str(h),
                horizontalalignment='center',
                transform=axs[i, e].transAxes)
        except:
            pass
        h += 1
plt.locator_params(axis="x", nbins=_nbins)
plt.savefig('kde_all_100.png')
plt.close()

### PLOTTO CON PLOTLY

In [ ]:
df_zo.columns

#### Plot  epoch singolo --> per diametri a piano campagna, Pds > 0

In [ ]:
# import plotly.graph_objects as go

# tracciato principale
x = df.POINT_X
y = df.POINT_Y

df0.dropna(inplace = True)
sizeref = 2.*max(df0['dave'])/(300)
# f = px.scatter(df0, x='long', y='lat', color='sector2000_50_str', size='dave')
# # senza colori per settore
#f = px.scatter(df0, x='long', y='lat', size='dave', hover_data=['sector2000_50','dbase'])

sizeref = 2.*max(df_zo.dbase)/300
f = px.scatter(df_zo[(df_zo.Pds>=0.00) & (df_zo.epoch==3)], x='long', y='lat', size='dbase', color = 'dbase', hover_data=['sector2000_50','Pds'])


f.update_traces(marker=dict(
    sizemode='area', sizeref=sizeref, line_width=2))

# f.add_trace(go.Scatter(x=df0.long, y=df0.lat, mode='markers', marker_size=df0.dave, marker=dict(
#     sizemode='area', sizeref=sizeref, line_width=2)))

# senza spessore punti
# f.add_trace(go.Scatter(x=df_zo[(df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['long'], y=df_zo[(
#     df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['lat'], mode='markers', marker=dict(size=0.5,color=2)))

# con spessore punti
# f.add_trace(go.Scatter(x=df_zo[df_zo['dbase'] > 0]['long'], y=df_zo[df_zo['dbase'] > 0]['lat'], mode='markers', marker_size=sizeref*10))

f.add_trace(go.Scatter(x=x, y=y))

f.add_trace(go.Scatter(x=vtrackup[:, 0], y=vtrackup[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown[:, 0], y=vtrackdown[:, 1]))

f.add_trace(go.Scatter(x=vtrackup500[:, 0], y=vtrackup500[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown500[:, 0], y=vtrackdown500[:, 1]))

f.add_trace(go.Scatter(x=vtrackup2000[:, 0], y=vtrackup2000[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown2000[:, 0], y=vtrackdown2000[:, 1]))




f.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)


f.update_layout(
    autosize=False,
    width=1100,
    height=800,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ), legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

f.show()

#### Plotto intere simulazioni solo subsidenze

In [ ]:
# import plotly.graph_objects as go

# tracciato principale
x = df.POINT_X
y = df.POINT_Y

df0.dropna(inplace = True)
sizeref = 2.*max(df0['dave'])/(300)
# f = px.scatter(df0, x='long', y='lat', color='sector2000_50_str', size='dave')
# # senza colori per settore
#f = px.scatter(df0, x='long', y='lat', size='dave', hover_data=['sector2000_50','dbase'])

sizeref = 2.*max(df_zo.dbase)/300
f = px.scatter(df_zo[(df_zo.Pds>=0.01) & (df_zo.Pds< 0.1)], x='long', y='lat', size='dbase', color = 'dbase', hover_data=['sector2000_50','Pds'])


f.update_traces(marker=dict(
    sizemode='area', sizeref=sizeref, line_width=2))

# f.add_trace(go.Scatter(x=df0.long, y=df0.lat, mode='markers', marker_size=df0.dave, marker=dict(
#     sizemode='area', sizeref=sizeref, line_width=2)))

# senza spessore punti
# f.add_trace(go.Scatter(x=df_zo[(df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['long'], y=df_zo[(
#     df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['lat'], mode='markers', marker=dict(size=0.5,color=2)))

# con spessore punti
# f.add_trace(go.Scatter(x=df_zo[df_zo['dbase'] > 0]['long'], y=df_zo[df_zo['dbase'] > 0]['lat'], mode='markers', marker_size=sizeref*10))

f.add_trace(go.Scatter(x=x, y=y))

f.add_trace(go.Scatter(x=vtrackup[:, 0], y=vtrackup[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown[:, 0], y=vtrackdown[:, 1]))

f.add_trace(go.Scatter(x=vtrackup500[:, 0], y=vtrackup500[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown500[:, 0], y=vtrackdown500[:, 1]))

f.add_trace(go.Scatter(x=vtrackup2000[:, 0], y=vtrackup2000[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown2000[:, 0], y=vtrackdown2000[:, 1]))




f.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)


f.update_layout(
    autosize=False,
    width=1100,
    height=800,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ), legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

f.show()

#### Plotto intere simulazioni doline + subsidenze - colore Pds 

In [ ]:
# import plotly.graph_objects as go

# tracciato principale
x = df.POINT_X
y = df.POINT_Y

df0.dropna(inplace = True)
sizeref = 2.*max(df0['dave'])/(300)
# f = px.scatter(df0, x='long', y='lat', color='sector2000_50_str', size='dave')
# # senza colori per settore
#f = px.scatter(df0, x='long', y='lat', size='dave', hover_data=['sector2000_50','dbase'])


# #  tutte le simulazioni solo doline
# sizeref = 2.*max(df_zo.dbase)/300
# f = px.scatter(df_zo[(df_zo.Pds>=0.1)], x='long', y='lat', size='dbase', color = 'dbase', hover_data=['sector2000_50','Pds'])

# #  tutte le simulazioni solo subsidenze
# sizeref = 2.*max(df_zo.dbase)/300
# f = px.scatter(df_zo[(df_zo.Pds>=0.01) & df_zo[(df_zo.Pds<0.1)], x='long', y='lat', size='dbase', color = 'dbase', hover_data=['sector2000_50','Pds'])

#  una simulazione Pds >= 0.01
sizeref = 2.*max(df_zo.dbase)/500
f = px.scatter(df_zo[(df_zo.Pds>=0.01) & ((df_zo.epoch==10) | (df_zo.epoch==20))], x='long', y='lat', size='dbase', color = 'Pds', hover_data=['sector2000_50','Pds'])


f.update_traces(marker=dict(
    sizemode='area', sizeref=sizeref, line_width=2))

# f.add_trace(go.Scatter(x=df0.long, y=df0.lat, mode='markers', marker_size=df0.dave, marker=dict(
#     sizemode='area', sizeref=sizeref, line_width=2)))

# senza spessore punti
# f.add_trace(go.Scatter(x=df_zo[(df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['long'], y=df_zo[(
#     df_zo['dbase'] > 0) & (df_zo.Pds > 0.01)]['lat'], mode='markers', marker=dict(size=0.5,color=2)))

# con spessore punti
# f.add_trace(go.Scatter(x=df_zo[df_zo['dbase'] > 0]['long'], y=df_zo[df_zo['dbase'] > 0]['lat'], mode='markers', marker_size=sizeref*10))

f.add_trace(go.Scatter(x=x, y=y))

f.add_trace(go.Scatter(x=vtrackup[:, 0], y=vtrackup[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown[:, 0], y=vtrackdown[:, 1]))

f.add_trace(go.Scatter(x=vtrackup500[:, 0], y=vtrackup500[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown500[:, 0], y=vtrackdown500[:, 1]))

f.add_trace(go.Scatter(x=vtrackup2000[:, 0], y=vtrackup2000[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown2000[:, 0], y=vtrackdown2000[:, 1]))




f.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)


f.update_layout(
    autosize=False,
    width=1100,
    height=800,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ), legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

f.show()

In [ ]:
# import plotly.graph_objects as go

# tracciato principale
x = df.POINT_X
y = df.POINT_Y

df0.dropna(inplace = True)
sizeref = 2.*max(df0['dave'])/(300)
# f = px.scatter(df0, x='long', y='lat', color='sector2000_50_str', size='dave')
# # senza colori per settore
#f = px.scatter(df0, x='long', y='lat', size='dave', hover_data=['sector2000_50','dbase'])

sizeref = 2.*max(df_zo.dbase)/300
f = px.scatter(df_zo_100[df_zo_100.epoch<10], x='long', y='lat', size='dbase', hover_data=['sector2000_50','Pds'])


f.update_traces(marker=dict(
    sizemode='area', sizeref=sizeref, line_width=2))

# f.add_trace(go.Scatter(x=df0.long, y=df0.lat, mode='markers', marker_size=df0.dave, marker=dict(
#     sizemode='area', sizeref=sizeref, line_width=2)))

# senza spessore punti
f.add_trace(go.Scatter(x=df_zo_100[(df_zo_100['dbase'] > 0) & (df_zo_100.Pds > 0)&(df_zo_100.epoch<10)]['long'], y=df_zo_100[(
    df_zo_100['dbase'] > 0) & (df_zo_100.Pds > 0)&(df_zo_100.epoch<10)]['lat'], mode='markers', marker=dict(size=8,color=2)))

# con spessore punti
# f.add_trace(go.Scatter(x=df_zo[df_zo['dbase'] > 0]['long'], y=df_zo[df_zo['dbase'] > 0]['lat'], mode='markers', marker_size=sizeref*10))

f.add_trace(go.Scatter(x=x, y=y))

f.add_trace(go.Scatter(x=vtrackup[:, 0], y=vtrackup[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown[:, 0], y=vtrackdown[:, 1]))

f.add_trace(go.Scatter(x=vtrackup500[:, 0], y=vtrackup500[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown500[:, 0], y=vtrackdown500[:, 1]))

f.add_trace(go.Scatter(x=vtrackup2000[:, 0], y=vtrackup2000[:, 1]))
f.add_trace(go.Scatter(x=vtrackdown2000[:, 0], y=vtrackdown2000[:, 1]))




f.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)


f.update_layout(
    autosize=False,
    width=1100,
    height=600,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ), legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

f.show()

In [ ]:
df_g50.columns

In [ ]:
df_g50

## CREO KARST STOCHASTICI

### creo colonne dei boundary 100m 200m e 2000m

###  CONVERSIONE IN COORDINATE UTM

In [ ]:
from pyproj import Proj

In [ ]:
x = tuple(df.POINT_X)
y = tuple(df.POINT_Y)

In [ ]:
myProj = Proj("+proj=utm +zone=38N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [ ]:
lon, lat = myProj(x, y, inverse=True)

In [ ]:


fig = px.scatter_mapbox(lat=lat, lon=lon, zoom=8, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.scatter_mapbox(lat=lat, lon=lon, zoom=7, height=300)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()